In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Notebook author:** emeinhardt@ucsd.edu

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Motivation" data-toc-modified-id="Motivation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Motivation</a></span><ul class="toc-item"><li><span><a href="#What-we-want-to-calculate" data-toc-modified-id="What-we-want-to-calculate-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>What we <em>want</em> to calculate</a></span></li><li><span><a href="#What-we-can-calculate" data-toc-modified-id="What-we-can-calculate-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>What we <em>can</em> calculate</a></span></li><li><span><a href="#The-structure-of-calculations" data-toc-modified-id="The-structure-of-calculations-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>The structure of calculations</a></span></li><li><span><a href="#The-structure-of-the-pipeline" data-toc-modified-id="The-structure-of-the-pipeline-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>The structure of the pipeline</a></span></li></ul></li><li><span><a href="#Requirements" data-toc-modified-id="Requirements-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Requirements</a></span><ul class="toc-item"><li><span><a href="#Python-environment" data-toc-modified-id="Python-environment-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Python environment</a></span></li><li><span><a href="#Hardware-/-runtime-expectations" data-toc-modified-id="Hardware-/-runtime-expectations-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Hardware / runtime expectations</a></span></li></ul></li><li><span><a href="#Todo" data-toc-modified-id="Todo-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Todo</a></span></li></ul></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Step-0:-Check-for-foundational-files" data-toc-modified-id="Step-0:-Check-for-foundational-files-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Step 0: Check for foundational files</a></span><ul class="toc-item"><li><span><a href="#Step-0a:-Check-for-gating-data" data-toc-modified-id="Step-0a:-Check-for-gating-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Step 0a: Check for gating data</a></span></li><li><span><a href="#Step-0b:-Check-for-transcribed-lexicons" data-toc-modified-id="Step-0b:-Check-for-transcribed-lexicons-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Step 0b: Check for transcribed lexicons</a></span></li><li><span><a href="#Step-0c:-Check-for-n-gram-contexts" data-toc-modified-id="Step-0c:-Check-for-n-gram-contexts-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Step 0c: Check for n-gram contexts</a></span></li><li><span><a href="#Step-0d:-Check-for-language-model(s)" data-toc-modified-id="Step-0d:-Check-for-language-model(s)-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Step 0d: Check for language model(s)</a></span></li></ul></li><li><span><a href="#Step-1:-Segment-inventory-alignment" data-toc-modified-id="Step-1:-Segment-inventory-alignment-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Step 1: Segment inventory alignment</a></span><ul class="toc-item"><li><span><a href="#Step-1a:-Define-inventory-alignment-projections" data-toc-modified-id="Step-1a:-Define-inventory-alignment-projections-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Step 1a: Define inventory alignment projections</a></span></li><li><span><a href="#Step-1b:-Apply-inventory-alignment-projections" data-toc-modified-id="Step-1b:-Apply-inventory-alignment-projections-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Step 1b: Apply inventory alignment projections</a></span><ul class="toc-item"><li><span><a href="#Check-for-projection-definitions" data-toc-modified-id="Check-for-projection-definitions-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Check for projection definitions</a></span></li><li><span><a href="#How-are-inventory-alignment-projections-actually-applied?" data-toc-modified-id="How-are-inventory-alignment-projections-actually-applied?-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>How are inventory alignment projections actually applied?</a></span></li><li><span><a href="#Apply-projection-definitions" data-toc-modified-id="Apply-projection-definitions-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Apply projection definitions</a></span></li></ul></li></ul></li><li><span><a href="#Step-2:-Generating-channel-and-(orthographic)-lexicon-distributions" data-toc-modified-id="Step-2:-Generating-channel-and-(orthographic)-lexicon-distributions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Step 2: Generating channel and (orthographic) lexicon distributions</a></span><ul class="toc-item"><li><span><a href="#Step-2a:-Generating-channel-distributions-and-associated-metadata" data-toc-modified-id="Step-2a:-Generating-channel-distributions-and-associated-metadata-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Step 2a: Generating channel distributions and associated metadata</a></span><ul class="toc-item"><li><span><a href="#Metadata" data-toc-modified-id="Metadata-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Channel-distributions" data-toc-modified-id="Channel-distributions-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Channel distributions</a></span></li></ul></li><li><span><a href="#Step-2b:-Generating-(contextual)-lexicon-distributions-(over-orthographic-vocabularies)" data-toc-modified-id="Step-2b:-Generating-(contextual)-lexicon-distributions-(over-orthographic-vocabularies)-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Step 2b: Generating (contextual) lexicon distributions (over orthographic vocabularies)</a></span></li></ul></li><li><span><a href="#Step-3:-Creating-combinable-models" data-toc-modified-id="Step-3:-Creating-combinable-models-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Step 3: Creating combinable models</a></span><ul class="toc-item"><li><span><a href="#Step-3a:-Filter-transcription-lexicons-to-only-include-words-that-can-be-modeled-by-a-given-channel-distribution" data-toc-modified-id="Step-3a:-Filter-transcription-lexicons-to-only-include-words-that-can-be-modeled-by-a-given-channel-distribution-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Step 3a: Filter transcription lexicons to only include words that can be modeled by a given channel distribution</a></span></li><li><span><a href="#Step-3b:-Filter-transcription-lexicons-to-only-include-words-that-are-in-a-language-model's-vocabulary" data-toc-modified-id="Step-3b:-Filter-transcription-lexicons-to-only-include-words-that-are-in-a-language-model's-vocabulary-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Step 3b: Filter transcription lexicons to only include words that are in a language model's vocabulary</a></span></li><li><span><a href="#Step-3c:-Filter-the-conditioning-events-of-channel-distributions-to-only-include-triphones-contained-in-a-transcription-lexicon's-segmental-wordform-list" data-toc-modified-id="Step-3c:-Filter-the-conditioning-events-of-channel-distributions-to-only-include-triphones-contained-in-a-transcription-lexicon's-segmental-wordform-list-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Step 3c: Filter the conditioning events of channel distributions to only include triphones contained in a transcription lexicon's segmental wordform list</a></span></li><li><span><a href="#Step-3d:-For-each-(filtered)-transcribed-lexicon-relation,-define-the-relevant-contextual-lexicon-distributions-over-orthographic-wordforms" data-toc-modified-id="Step-3d:-For-each-(filtered)-transcribed-lexicon-relation,-define-the-relevant-contextual-lexicon-distributions-over-orthographic-wordforms-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Step 3d: For each (filtered) transcribed lexicon relation, define the relevant contextual lexicon distributions over orthographic wordforms</a></span></li><li><span><a href="#Step-3e:-For-each-(filtered)-transcribed-lexicon-relation,-define-a-conditional-distribution-on-segmental-wordforms-given-an-orthographic-wordform" data-toc-modified-id="Step-3e:-For-each-(filtered)-transcribed-lexicon-relation,-define-a-conditional-distribution-on-segmental-wordforms-given-an-orthographic-wordform-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Step 3e: For each (filtered) transcribed lexicon relation, define a conditional distribution on segmental wordforms given an orthographic wordform</a></span></li></ul></li><li><span><a href="#Step-4:-Pre-calculate-remaining-forward-model-components-and-meta-data" data-toc-modified-id="Step-4:-Pre-calculate-remaining-forward-model-components-and-meta-data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Step 4: Pre-calculate remaining forward model components and meta-data</a></span><ul class="toc-item"><li><span><a href="#Step-4a:-Generate-triphone-lexicon-distributions-for-every-triphone-channel-model" data-toc-modified-id="Step-4a:-Generate-triphone-lexicon-distributions-for-every-triphone-channel-model-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Step 4a: Generate triphone lexicon distributions for every triphone channel model</a></span></li><li><span><a href="#Step-4b:-Pre-calculate-prefix-relation,-$k$-cousins,-and-$k$-spheres-for-each-segmental-lexicon" data-toc-modified-id="Step-4b:-Pre-calculate-prefix-relation,-$k$-cousins,-and-$k$-spheres-for-each-segmental-lexicon-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Step 4b: Pre-calculate prefix relation, $k$-cousins, and $k$-spheres for each segmental lexicon</a></span></li><li><span><a href="#Step-4c:-Calculate-the-marginal-probability-$p(W|C)$-of-each-segmental-wordform-$w$-given-$n$-gram-contexts-$C$" data-toc-modified-id="Step-4c:-Calculate-the-marginal-probability-$p(W|C)$-of-each-segmental-wordform-$w$-given-$n$-gram-contexts-$C$-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Step 4c: Calculate the marginal probability $p(W|C)$ of each segmental wordform $w$ given $n$-gram contexts $C$</a></span></li><li><span><a href="#Step-4d:-Define-channel-distributions-on-a-set-of-segmental-wordforms(+prefixes)" data-toc-modified-id="Step-4d:-Define-channel-distributions-on-a-set-of-segmental-wordforms(+prefixes)-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Step 4d: Define channel distributions on a set of segmental wordforms(+prefixes)</a></span></li></ul></li><li><span><a href="#Step-5:-Calculate-posterior-probabilities" data-toc-modified-id="Step-5:-Calculate-posterior-probabilities-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Step 5: Calculate posterior probabilities</a></span><ul class="toc-item"><li><span><a href="#Step-5a:-Calculate-$p(V|W,-C)$" data-toc-modified-id="Step-5a:-Calculate-$p(V|W,-C)$-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Step 5a: Calculate $p(V|W, C)$</a></span></li><li><span><a href="#Step-5b:-Calculate-$p(\hat{X}_0^f|X_0^f,-C)$" data-toc-modified-id="Step-5b:-Calculate-$p(\hat{X}_0^f|X_0^f,-C)$-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Step 5b: Calculate $p(\hat{X}_0^f|X_0^f, C)$</a></span></li><li><span><a href="#Step-5c:-Calculate-$p(\hat{V}-=-v^*|-V-=-v^*,-C)$" data-toc-modified-id="Step-5c:-Calculate-$p(\hat{V}-=-v^*|-V-=-v^*,-C)$-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Step 5c: Calculate $p(\hat{V} = v^*| V = v^*, C)$</a></span></li></ul></li><li><span><a href="#Step-6:-Generating-analysis-measures" data-toc-modified-id="Step-6:-Generating-analysis-measures-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Step 6: Generating analysis measures</a></span></li></ul></div>

# Overview

This notebook describes the processing pipeline from 
 - gating data
 - transcribed lexicon
 - a language model and (possibly empty) n-gram contexts

to 
 - channel distribution
 - lexicon distribution(s) (distributions over wordforms)
 - expected posterior distribution over intended wordform given what has been produced of what was intended.
 
 
It describes what happens at each step, checks some pre- and post-conditions, describes what you, the user must do (if anything), and scripts some commands to automatically do the necessary processing.

## Motivation

There are about one-and-a-half calculations that this processing pipeline enables.

Let 
 - $C = \{c_0, c_1 ... c_{n-1}\}$ denote a set of $n$-gram contexts (sequences of $\leq (n-1)$ orthographic wordforms) drawn from a speech corpus (e.g. Buckeye or Switchboard).
 - $V = \{v_0, v_1 ... v_V\}$ denote a set of orthographic wordforms (a 'vocabulary') drawn from a speech corpus (the same corpus $C$ is drawn from).
 - $W = \{w_0, w_1 ... w_W\}$ denote a set of segmental wordforms ('transcribed wordforms'). Each element $w \in W$ consists of a sequence $x_0 x_1 ... x_f = x_0^f$ of segments ('phonemes') drawn from a set of symbols $\Sigma_x$.
 
(All sets are finite, unless otherwise noted.)

 - A language model describes $p(V|C)$.
 - A lexical database or speech corpus describes a relation $r$ between orthographic wordforms $V$ and segmental wordforms $W$, where $(v,w) \in r$ iff $v$ can be produced as $w$.
 - Let $p(W|V)$ be defined as follows: $p(w|v) = \frac{1}{r_{v}}$, where $r_v = |\{w' | (v, w') \in r\}|$
 - Given that the $i$th segment that a speaker intended to produce is $x_i$, a triphone channel model describes $p(Y_i|x_{i-1}, x_i; x_{i+1})$, a distribution over received/perceived segments $\Sigma_y$. This is estimated from diphone gating data. Note that it doesn't permit modeling insertions or deletions.
 - $p(Y_0^f|W) = p(Y_0^f|X_0^f)$ is a (channel) distribution over perceived (segmental) wordforms given an intended (segmental) wordform. It is completely defined by a choice of $p(W|V)$ and a choice of $p(Y_i|x_{i-1}, x_i; x_{i+1})$.

The *forward model*, then defines a distribution $p(Y_0^f, X_0^f, V|C) = p(Y_0^f|X_0^f)p(X_0^f|V)p(V|C)$.

### What we *want* to calculate

We are interested in calculating the *expected posterior of a listener* over what orthographic wordform the speaker intended given what the speaker actually intended (taking the expectation over the speaker's actual intended segmental wordform, the perceived segmental wordform, and the listener's estimate of the speaker's intended segmental wordform):

$$p(\hat{V} = v^*|V = v^*, c) = \sum\limits_{x_0^{*f}, y_0^f, x_0^{'f}} p(\hat{V} = v^*, \hat{X}_0^f = x_0^{'f}, y_0^f, X_0^f = x_0^{*f}|V = v^*, c)$$

$$p(\hat{V} = v^*|V = v^*, c) = \sum\limits_{x_0^{*f}, y_0^f, x_0^{'f}} p(\hat{V} = v^*|\hat{X}_0^f = x_0^{'f}, c) p(\hat{X}_0^f = x_0^{'f}|Y_0^f = y_0^f, c) p(Y_0^f = y_0^f | X_0^f = x_0^{*f})p(X_0^f = x_0^{*f}|V = v^*)$$

where 

1. $p(\hat{X}_0^f = x_0^{'f}|Y_0^f = y_0^f, c) = \frac{p(y_0^f|x_0^{'f})p(x_0^{'f}|c)}{p(y_0^f | c)}$
2. $p(x_0^{'f}|c) = \sum\limits_{v'} p(x_0^{'f}|v')p(v'|c)$
3. $p(y_0^f| c) = \sum\limits_{v', x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|v')p(v'|c) = \sum\limits_{x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|c)$
4. $p(\hat{V} = v^*|\hat{X}_0^f = x_0^{'f}, c) = \frac{p(x_0^{'f}|v^*)p(v^*|c)}{p(x_0^{'f}|c)}$ 


### What we *can* calculate

 1. Unfortunately, because of the enormous size of $Y_0^f$, exact marginalization over all $y_0^f$ is *not* feasible.
 2. Fortunately, because, for any given $x_0^{*f}$, the fraction of $Y_0^f$ with non-negligible mass in $p(Y_0^f|x_0^{*f})$ is small, we can construct a Monte Carlo estimate of 
 $$p(\hat{X}_0^f = x_0^{'f}|x_0^{*f}, c) = \sum\limits_{y_0^f} p(\hat{X}_0^f = x_0^{'f}|y_0^f, c) p(y_0^f|x_0^{*f})$$
 as
 $$\hat{p}(\hat{X}_0^f = x_0^{'f}|x_0^{*f}, c) = \frac{1}{n} \sum\limits_{y_0^f \in S} p(\hat{X}_0^f = x_0^{'f}|y_0^f, c)$$
 where $S = $ a set of $n$ samples from $p(Y_0^f|x_0^{*f})$. In practice an $n \approx 50$ seems to result in estimates that are within $10^{-6}$ of the true estimate. 
 
 3. Unfortunately, even with this approximation $p(\hat{X}_0^f|X_0^{*f}, c)$ has about $10^8 - 10^9$ entries: too many to feasibly calculate exactly, especially across *all choices of $c$* as well.
 4. However, because of the relative dispersion of wordforms, the relatively low overall rate of channel noise, and the information provided by sentential context, the fraction of $\hat{X}_0^f$ assigned non-negligible mass in $p(\hat{X}_0^f|x_0^{*f}, c)$ for any given $x_0^{*f}$ is relatively small, and largely concentrated on $x_0^{'f}$ that are within $k$ edits (substitutions) of $x_0^{*f}$ for small $k$. Accordingly, we can get an arbitrarily good approximation $\hat{p}^{k}$ of $p(\hat{X}_0^f|x_0^{*f}, c)$ by 
  - choosing an arbitrarily small threshold $\epsilon$
  - calculating
$$p^k = \{p(\hat{X}_0^f = x_0^{'f}|x_0^{*f}, c) | x_0^{'f} \text{ is within } k \text{ edits of }x_0^{*f}\}$$ for progressively increasing $k$, stopping when $1 - \sum p_i^k \leq \epsilon$ and assigning $0$ probability to all $x_0^{'f}$ not associated with $p^k$.

Note that this means that if we choose the same $\epsilon$ for all $(x_0^{*f}, c)$ pairs, some segmental wordforms $x_0^f$ will have approximations involving higher or lower $k$ values than others, but that all will have distributions that are at least as accurate as some same minimum (determined by $\epsilon$).

### The structure of calculations

$$p(\hat{V} = v^*|V = v^*, c) = \sum\limits_{x_0^{*f}, x_0^{'f}} p(\hat{V} = v^*|\hat{X}_0^f = x_0^{'f}, c) p(\hat{X}_0^f = x_0^{'f}| X_0^f = x_0^{*f}, c) p(X_0^f = x_0^{*f}|V = v^*)$$

so $$p(\hat{V} = v^*|V = v^*, c) \approx \sum\limits_{x_0^{*f}, x_0^{'f}. d(x_0^{*f}, x_0^{'f}) \leq k} p(\hat{V} = v^*|\hat{X}_0^f = x_0^{'f}, c) \hat{p}(\hat{X}_0^f = x_0^{'f}| X_0^f = x_0^{*f}, c) p(X_0^f = x_0^{*f}|V = v^*)$$

We want to have the following distributions pre-computed (or as close to that as possible) as `numpy` arrays
 1. $p(W|V) = p(X_0^f|V)$
 2. $\hat{p}(\hat{X}_0^f|X_0^f, C)$
 3. $p(Y_0^f|W)$
 4. $p(W|C)$
 5. $p(V|W, C)$
 
so that we can efficiently calculate $p(\hat{V} = v^*| V = v^*, c)$ for all pairs of $(v^*, c)$ and
where 
 - $p(\hat{V} = v^*| V = v^*, c)$
 - $\hat{p}(\hat{X}_0^f|X_0^f, C)$, and 
 - $p(V|W,C)$ 
 
involve decreasing amounts of non-trivial computation.

### The structure of the pipeline

Given foundational files:
 - a language model $m$ over some orthographic vocabulary $V_m$
 - $n$-gram contexts $C$ taken from one or more speech corpora
 - a target orthographic vocabulary $V$ taken from each of the speech corpora that contexts are taken from
 - diphone gating data
 - one or more transcription relations $r$
 
the first step in the processing pipeline involves aligning segmental inventories of gating data and transcription relations. Once that's been done, we can define triphone channel distributions on transcription-relation aligned gating data that can be applied to gating-data aligned transcribed wordforms. We also need to define $p(V_m|C)$ for each choice of $C$.

Next, we create mutually combinable versions of transcription relations, channel distributions, and distributions over orthographic vocabularies - e.g.
 - we need to remove words from each $r$ that contain triphones that can't be modeled by the aligned triphone channel distribution
 - we need to remove words from each $r$ that aren't in the language model vocabulary $V_m$
 - $p(V_m|C)$ needs to be projected down to remove orthographic words that aren't in transcription relations and context sets $C$ need to be scrubbed of contexts containing orthographic wordforms not in the language model

With all the atomic components of the model(/each possible combined model) constructed, we then pre-calculate remaining components of the forward model(s). Finally, we calculate components of the posterior. (The separation of this last step from *everything previous* facilitates parallel computation.)

## Requirements

### Python environment
This repository was developed using Python 3.6 with the following package requirements (upstream dependencies not included):
 - **Jupyter/notebook related packages**: `jupyter` `jupyter_contrib_nbextensions` `papermill`
    - Notable notebook extensions used include `ExecuteTime` and `Table of Contents (2)`.
 - **Numeric computing and data processing**: `scipy` `numpy` `pandas` `tqdm` `joblib` `numba` `sparse` `pytorch` `torchvision`
 - **Plotting**: `matplotlib` `plotnine`
 - **Misc**: `funcy` `more_itertools`

See `dev_environment.sh` for `conda` and `pip` commands to create an environment that has all of these packages. (Not all are available on conda, and not all of those that are available on conda are available from the main channel.)


### Hardware / runtime expectations

The last machine this repository was developed on (`wittgenstein`) has 32 cores. `joblib` is used extensively to parallelize data processing. On this machine, using `joblib` and often using nearly all of those cores:
1. Step 1 takes about a minute.
2. Step 2 takes about 3 hours.
3. Step 3 takes about 15-20 minutes.
4. Step 4 takes about 4.5 hours.

To run these scripts comfortably and without modification, you will need about 120GB of memory and ≈250 GB of hard disk space. (Little or no attempt has been made to compress or avoid unnecessary file outputs except insofar as it creates representations that lead to usefully smaller memory loads or facilitate matrix-based compuations.)

## Todo

0. **Extensibility**: Step 3b (filtering transcribed lexicons against language model vocabularies) uses output filenames that won't scale if any transcription lexicon is used with more than one language model. (This also affects 3c.)
1. **Extensibility/Modularity/Maintainability**: Every notebook that depends on the behavior or output of another notebook being a certain way should have that assumption flagged at the top of the notebook. 
   - **Most common example**: one of notebook $B$'s arguments is a *directory path* $d$, where it expects to find a specific set of files with certain fixed filenames (or with filenames that are derived in some way from one of $B$'s arguments - often $d$); this assumption is predicated on the behavior of some notebook $A$ earlier in the pipeline producing exactly some set of files in a common directory all with certain filenames.
2. **Portability/Reproducibility**: For every file that this repository depends on that *isn't* tracked by the repository (e.g. processed versions of swbd2003, Buckeye, etc.), there should be *something* (e.g. a cell in this script) that lets the user identify where those files are located, and then said something copies them to wherever this repository is expecting to find them.
3. **Portability/Reproducibility**: Check for and remove absolute paths in this and other files.
4. **Portability/Reproducibility**: For platform independence (read: supporting windows users, I guess?) use `path.join` instead of manually choosing directory slashes...
5. **Documentation**: 
   1. Make sure motivation section is up to date.
   2. Math-y documentation in channel distribution and posterior distribution notebooks probably needs to be updated / at least have notation overhauled.
   3. Go through notebooks used here and make sure `Overview` cells are accurate.
   4. Go through notebooks used here and make sure Papermill-related `Usage` cells are filled in / accurate.

# Imports

In [2]:
import papermill as pm

In [3]:
from tqdm import tqdm

In [4]:
from os import getcwd, chdir, listdir, path, mkdir, makedirs

import json
import csv

In [5]:
from collections import OrderedDict

In [6]:
from itertools import product
from boilerplate import ensure_dir_exists, union

In [7]:
repo_dir = getcwd()
repo_dir

'/mnt/cube/home/AD/emeinhar/wr'

In [8]:
repo_contents_0 = listdir()
repo_contents_0

['probdist.py',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0',
 'Generate triphone lexicon distribution from channel model.ipynb',
 'swbd2003_contexts.txt',
 'boilerplate.py',
 'GD_AmE-diphones - LTR_Buckeye alignment application to LTR_Buckeye.ipynb',
 'LTR_Buckeye',
 '.gitignore',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1',
 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed',
 'Run n-phone analysis of gating data.ipynb',
 'GD_AmE_destressed_aligned_w_LTR_Buckeye',
 'Calculate segmental wordform distribution given corpus contexts.ipynb',
 '__pycache__',
 'GD_AmE-diphones - LTR_CMU_destressed alignment application to LTR_CMU_destressed.ipynb',
 'GD_AmE-diphones - LTR_CMU_destressed alignment definition.ipynb',
 'LD_Fisher_vocab_in_Buckeye_contexts',
 'GD_AmE-diph

# Step 0: Check for foundational files

**NOTE:**
 - I assume all relevant transcriptions have been converted to Unicode IPA characters and that segment sequences have `.` as a separator. For each data source used here, the IPA alignment step is documented in a GitHub repository elsewhere. (While I don't *think* any script here depends on use of Unicode IPA symbols, I haven't - and won't - test that idea, and it is *absolutely required* that contiguous sequences of segments be separated by `.` in data files.)
 - Where language models and n-gram contexts (drawn from speech corpora) are referenced, each of these is assumed to have come from as is from other GitHub repositories.

The four steps below verify that foundational files assumed to be present by downstream notebooks are, in fact, present in the repository directory. If for some reason those files are not present, the processing pipeline will be aborted.

## Step 0a: Check for gating data

In [9]:
AmE_gating_data_dir = 'GD_AmE'
AmE_gating_data_fn = 'AmE-diphones-IPA-annotated-columns.csv'
AmE_GD_fp = path.join(AmE_gating_data_dir, AmE_gating_data_fn)
assert path.exists(AmE_gating_data_dir), 'Gating data directory {0} does not exist.'.format(AmE_gating_data_dir)
assert path.exists(AmE_GD_fp), 'Gating data file {0} does not exist.'.format(AmE_GD_fp)

The processed gating data used here come from 
 - https://github.com/emeinhardt/wmc2014-ipa
 
See those repositories for information on how they were produced.

## Step 0b: Check for transcribed lexicons

 - Each transcribed lexicon `LEXNAME` should be in a folder (e.g. `LTR_LEXNAME`) containing a file `LTR_LEXNAME.tsv`. For documentation purposes, the source file and a notebook documenting the production of the `.tsv` file should, if practicable be included in the folder as well.
   - A transcribed lexicon `LTR_....tsv` file should have two columns: `Orthographic_Wordform` and `Transcription`.
   - NB: The `LTR_` prefix on transcribed lexicon data files and containing folders is simply a convention for organization and readability, but is not required or expected by any file or script in this repository.

The assertions in the code below will only succeed if step 1 is complete for all transcribed lexicons listed for checking below.

In [10]:
newdic_destressed_ltr_folder = 'LTR_newdic_destressed'
cmu_destressed_ltr_folder = 'LTR_CMU_destressed'
cmu_stressed_ltr_folder = 'LTR_CMU_stressed'
buckeye_ltr_folder = 'LTR_Buckeye'
# nxt_swbd_ltr_folder = 

LTR_folders = (newdic_destressed_ltr_folder, cmu_destressed_ltr_folder, cmu_stressed_ltr_folder, buckeye_ltr_folder)
LTR_folders_to_process = (newdic_destressed_ltr_folder, cmu_destressed_ltr_folder, buckeye_ltr_folder)

for dirname in tqdm(LTR_folders_to_process):
    assert path.exists(dirname), 'Transcribed lexicon directory {0} not found in repo directory'.format(dirname)
    fname = path.join(dirname, dirname + '.tsv')
    assert path.exists(fname), 'Transcribed lexicon {0} not found in repo directory'.format(fname)

100%|██████████| 3/3 [00:00<00:00, 66.41it/s]


**How are transcribed lexicon relations made?**

Each was created by processing a transcription source (a lexical database, an annotated corpus, etc.). The processing step is described in other repositories:
 - https://github.com/emeinhardt/newdic-nettalk-ipa
 - https://github.com/emeinhardt/cmu-ipa
 - https://github.com/emeinhardt/buckeye-lm
 - https://github.com/emeinhardt/switchboard-lm

Given the processed outputs of these repositories, the `Making a Transcribed Lexicon Relation - <LEXNAME>.ipynb` notebook in each `LTR_LEXNAME` folder describes how the homogeneous `.tsv` files downstream steps are created.

In [11]:
for dirname in LTR_folders_to_process:
    listdir(dirname)

['LTR_newdic_destressed.tsv',
 'Making a Transcribed Lexicon Relation - newdic_destressed.ipynb',
 'newdic_IPA.tsv',
 '.ipynb_checkpoints']

['Making a Transcribed Lexicon Relation - CMU_destressed.ipynb',
 'LTR_CMU_destressed.pW_V.json',
 'LTR_CMU_destressed.tsv',
 '.ipynb_checkpoints',
 'LTR_CMU_destressed.pW_V.npz',
 'cmudict-0.7b_IPA_destressed.tsv',
 'LTR_CMU_destressed_Orthographic_Wordforms.txt',
 'LTR_CMU_destressed_Transcriptions.txt']

['LTR_Buckeye.tsv',
 'buckeye_words_analysis_relation.json',
 '.ipynb_checkpoints',
 'Making a Transcribed Lexicon Relation - Buckeye.ipynb']

## Step 0c: Check for n-gram contexts

In [12]:
buckeye_contexts = 'buckeye_contexts.txt'
swbd2003_contexts = 'swbd2003_contexts.txt'

contexts = (buckeye_contexts, swbd2003_contexts)

for c_fn in contexts:
    assert path.exists(c_fn), "N-gram contexts file {0} does not exist.".format(c_fn)

The n-gram context files are taken from
 - https://github.com/emeinhardt/buckeye-lm
 - https://github.com/emeinhardt/switchboard-lm
 
See those repositories for more information on how the contexts were extracted. (*NB*: The context files are not included in this repository both to avoid duplication and because of licensing restrictions: to recreate these contexts, you will need access to your own copy of the Buckeye and Switchboard corpora.)

## Step 0d: Check for language model(s)

In [13]:
fisher_lm_dir = 'LM_Fisher'
fisher_lm_fn = 'fisher_utterances_main_4gram.mmap'
fisher_lm_fp = path.join(fisher_lm_dir, fisher_lm_fn)

fisher_lm_vocab_fn = 'fisher_vocabulary_main.txt'
fisher_lm_vocab_fp = path.join(fisher_lm_dir, fisher_lm_vocab_fn)

assert path.exists(fisher_lm_fp), 'Language model {0} not found'.format(fisher_lm_fp)
assert path.exists(fisher_lm_vocab_fp), 'Language model vocabulary {0} not found'.format(fisher_lm_vocab_fp)

fisher_lm_fps = {'lm':fisher_lm_fp, 
                 'vocab':fisher_lm_vocab_fp}

The (memory mapped) 4-gram language model is copied from the output of this repository:
 - https://github.com/emeinhardt/fisher-lm
 
See that repository for more information. (*NB* Again, the language model file is not included in this repository both to avoid duplication and because of licensing restrictions: to recreate these contexts, you will need access to your own copy of the Buckeye and Switchboard corpora.

# Step 1: Segment inventory alignment

## Step 1a: Define inventory alignment projections

The segment inventory of any given transcribed lexicon and the segment inventory of the gating data often do not line up. For the gating data to be usefully applied to a given lexicon of transcriptions, the strings in the (segmental) lexicon must contain only segments found in the gating data stimuli inventory.

To ensure this happens, the notebook `Gating Data - Transcription Lexicon Alignment Maker.ipynb` 
 - takes as inputs 
     - a transcribed lexicon file path and a gating data file path
     - a lexicon projection file path and a gating data projection file path
 - identifies the inventories of each and what symbols are relatively unique to the lexicon and the gating data
 - produces 
   - *a Jupyter notebook* for **you to open and finish by defining two projection functions** (i.e. Python dictionaries) to be applied to strings in the transcribed lexicon and to the gating data (one function for each). When you finish doing this (and set an export flag in the notebook to True and run the remainder of the notebook), this notebook will produce
     - two *.json files storing these projections* according to the previously provided output file paths.

The cell below will clear all existing alignment folders created using the code in this subsection:

In [13]:
# %rm -rf LTR*_aligned_w_*
# %rm -rf *" alignment definition"*

The cell below will only succeed if the American English gating data of Warner, McQueen, and Cutler (2014) is contained in the repo directory with a particular directory and filename.

In [14]:
gating_data_folder = 'GD_AmE'
gating_data_fn = 'AmE-diphones-IPA-annotated-columns.csv'
gating_data_fp = path.join(gating_data_folder, gating_data_fn)

assert path.exists(gating_data_folder), 'AmE gating data folder {0} not found in repo directory'.format(gating_data_folder)
assert path.exists(gating_data_fp), 'AmE gating data {0} not found in repo directory'.format(gating_data_fp)

The third cell below will create a notebook for alignment projection definitions for each of the transcribed lexicons from the previous step and the AmE gating data.

In [15]:
def removeExtension(fp):
    dir_name = path.dirname(fp)
    file_name = path.basename(fp)
    ext = file_name.split('.')[-1]
    rest = '.'.join( file_name.split('.')[:-1] )
    return path.join(dir_name, rest)

In [16]:
alignment_arg_bundles = []
for LTR_dirname in LTR_folders_to_process:
    LTR_fn = LTR_dirname + '.tsv'
    LTR_fp = path.join(LTR_dirname, LTR_fn)
    
    nb_output_name = 'GD_AmE-diphones - ' + LTR_dirname + ' alignment definition' + '.ipynb'
    my_g = gating_data_fp
    my_l = LTR_fp
    my_s = 'destressed'
    
    gd_alignment_dn = 'GD_AmE_' + my_s + '_' + 'aligned_w_' + LTR_dirname
    gd_alignment_fn = 'alignment_of_' + removeExtension(gating_data_fn) + '_w_' + LTR_dirname + '.json'
    gd_alignment_fp = path.join(gd_alignment_dn, gd_alignment_fn)
    if not path.exists(gd_alignment_dn):
        makedirs(gd_alignment_dn)
    my_gp = gd_alignment_fp
    
    ltr_alignment_dn = LTR_dirname + '_aligned_w_' + 'GD_AmE_' + my_s
    ltr_alignment_fn = 'alignment_of_' + LTR_dirname + '_w_' + removeExtension(gating_data_fn) + '.json'
    ltr_alignment_fp = path.join(ltr_alignment_dn, ltr_alignment_fn)
    if not path.exists(ltr_alignment_dn):
        makedirs(ltr_alignment_dn)
    my_lp = ltr_alignment_fp
    
    
    my_arg_bundle = OrderedDict({
        'LTR_dirname':LTR_dirname,
        'LTR_fn':LTR_fn,
        'LTR_fp':LTR_fp,
        'gd_alignment_dn':gd_alignment_dn,
        'gd_alignment_fn':gd_alignment_fn,
        'gd_alignment_fp':gd_alignment_fp,
        'ltr_alignment_dn':ltr_alignment_dn,
        'ltr_alignment_fn':ltr_alignment_fn,
        'ltr_alignment_fp':ltr_alignment_fp,
        'align_def_nb_output_name':nb_output_name,
        'my_g':my_g,
        'my_l':my_l,
        'my_s':my_s,
        'my_gp':my_gp,
        'my_lp':my_lp,
    })
    alignment_arg_bundles.append(my_arg_bundle)

In [17]:
# takes ~30s on wittgenstein
for arg_bundle in tqdm(alignment_arg_bundles):
    nb = pm.execute_notebook(
        'Gating Data - Transcription Lexicon Alignment Maker.ipynb',
        arg_bundle['align_def_nb_output_name'],
        parameters=dict(g = arg_bundle['my_g'], 
                        l = arg_bundle['my_l'], 
                        s = arg_bundle['my_s'], 
                        gp = arg_bundle['my_gp'], 
                        lp = arg_bundle['my_lp'])
    )
#     pm.execute_notebook(
#        'Gating Data - Transcription Lexicon Alignment Maker.ipynb',
#        nb_output_name,
#        parameters=dict(g = my_g, l = my_l, s = my_s, gp = my_gp, lp = my_lp)
#     )
    print("Finished creating alignment definition notebook '{0}'.\nOpen and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').\n".format(arg_bundle['align_def_nb_output_name']))

  0%|          | 0/3 [00:00<?, ?it/s]Input Notebook:  Gating Data - Transcription Lexicon Alignment Maker.ipynb
Output Notebook: GD_AmE-diphones - LTR_newdic_destressed alignment definition.ipynb

 33%|███▎      | 1/3 [00:10<00:20, 10.07s/it]s]

Finished creating alignment definition notebook 'GD_AmE-diphones - LTR_newdic_destressed alignment definition.ipynb'.
Open and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').



Input Notebook:  Gating Data - Transcription Lexicon Alignment Maker.ipynb
Output Notebook: GD_AmE-diphones - LTR_CMU_destressed alignment definition.ipynb

 67%|██████▋   | 2/3 [00:20<00:10, 10.08s/it]s]

Finished creating alignment definition notebook 'GD_AmE-diphones - LTR_CMU_destressed alignment definition.ipynb'.
Open and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').



Input Notebook:  Gating Data - Transcription Lexicon Alignment Maker.ipynb
Output Notebook: GD_AmE-diphones - LTR_Buckeye alignment definition.ipynb

100%|██████████| 3/3 [00:29<00:00,  9.73s/it]s]

Finished creating alignment definition notebook 'GD_AmE-diphones - LTR_Buckeye alignment definition.ipynb'.
Open and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').



## Step 1b: Apply inventory alignment projections

The cell below will clear all existing alignment folders created using the code in this subsection:

In [18]:
# %rm -rf *" alignment application "*

### Check for projection definitions

The cell below will succeed if you have run each of the previously produced notebooks correctly and produced a projection mapping file.

In [17]:
for arg_bundle in alignment_arg_bundles:
    args = arg_bundle
    assert path.exists(args['gd_alignment_fp']), 'Gating data alignment projection mapping not found:\n\t{0}'.format(args['gd_alignment_fp'])
    assert path.exists(args['ltr_alignment_fp']), 'Transcribed lexicon data alignment projection mapping not found:\n\t{0}'.format(args['ltr_alignment_fp'])

### How are inventory alignment projections actually applied?

See `Align transcriptions.ipynb`.

### Apply projection definitions

The cell below applies each pair of alignment projections to each matched pair of gating data and transcribed lexicon choice:

In [18]:
for arg_bundle in alignment_arg_bundles:
    args = arg_bundle
    LTR_fn = args['LTR_fn']
    
#     my_pg = args['my_gp']
#     my_g = args['my_g']
    my_o_fn = 'GD_AmE-diphones' + '_aligned_w_' + removeExtension(LTR_fn) + '.tsv'
    my_og = path.join(args['gd_alignment_dn'], my_o_fn)
    args['align_apply_gd_nb_output_name'] = 'GD_AmE-diphones - ' + removeExtension(LTR_fn) + ' alignment application to ' + 'AmE-diphones' + '.ipynb'
    args['my_og'] = my_og
    
#     my_pl = args['my_lp']
#     my_l = args['my_l']
    my_o_fn = removeExtension(LTR_fn) + '_aligned_w_' + 'GD_AmE-diphones' + '.tsv'
    my_ol = path.join(args['ltr_alignment_dn'], my_o_fn)
    args['align_apply_ltr_nb_output_name'] = 'GD_AmE-diphones - ' + removeExtension(LTR_fn) + ' alignment application to ' + removeExtension(LTR_fn) + '.ipynb'
    args['my_ol'] = my_ol

In [21]:
# takes ~45s on wittgenstein
for arg_bundle in alignment_arg_bundles:
    args = arg_bundle
#     LTR_fn = args['LTR_fn']
    
    my_pg = args['my_gp']
    my_g = args['my_g']
#     my_o_fn = 'GD_AmE-diphones' + '_aligned_w_' + removeExtension(LTR_fn) + '.tsv'
#     my_og = path.join(args['gd_alignment_dn'], my_o_fn)
#     args['align_apply_gd_nb_output_name'] = 'GD_AmE-diphones - ' + removeExtension(LTR_fn) + ' alignment application to ' + 'AmE-diphones' + '.ipynb'
#     args['my_og'] = my_og
    my_og = args['my_og']
    print("Creating notebook '{0}' w/ args p, g, o = \n\t{1}\n\t{2}\n\t{3}".format(args['align_apply_gd_nb_output_name'], my_pg, my_g, my_og))
    nb = pm.execute_notebook(
        'Align transcriptions.ipynb',
        args['align_apply_gd_nb_output_name'],
        parameters=dict(p = my_pg,
                        g = my_g,
                        o = my_og)
    )
    print('Finished applying alignment projection\n\tp = {0}\nto\n\tg = {1}\nResult saved to\n\t{2}'.format(my_pg, my_g, my_og))
    print(' ')
    
    my_pl = args['my_lp']
    my_l = args['my_l']
#     my_o_fn = removeExtension(LTR_fn) + '_aligned_w_' + 'GD_AmE-diphones' + '.tsv'
#     my_ol = path.join(args['ltr_alignment_dn'], my_o_fn)
#     args['align_apply_ltr_nb_output_name'] = 'GD_AmE-diphones - ' + removeExtension(LTR_fn) + ' alignment application to ' + removeExtension(LTR_fn) + '.ipynb'
#     args['my_ol'] = my_ol
    my_ol = args['my_ol']
    print('Creating notebook {0} w/ args p, g, o = \n\t{1}\n\t{2}\n\t{3}'.format(args['align_apply_ltr_nb_output_name'], my_pg, my_l, my_ol))
    nb = pm.execute_notebook(
        'Align transcriptions.ipynb',
        args['align_apply_ltr_nb_output_name'],
        parameters=dict(p = my_pl,
                        l = my_l,
                        o = my_ol)
    )
    print('Finished applying alignment projection\n\tp = {0}\nto\n\tl = {1}\nResult saved to\n\t{2}'.format(my_pl, my_l, my_ol))
    print('\n')

Creating notebook 'GD_AmE-diphones - LTR_newdic_destressed alignment application to AmE-diphones.ipynb' w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_newdic_destressed/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_newdic_destressed.json
	GD_AmE/AmE-diphones-IPA-annotated-columns.csv
	GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv


Input Notebook:  Align transcriptions.ipynb
Output Notebook: GD_AmE-diphones - LTR_newdic_destressed alignment application to AmE-diphones.ipynb
100%|██████████| 64/64 [00:07<00:00,  8.05it/s]


Finished applying alignment projection
	p = GD_AmE_destressed_aligned_w_LTR_newdic_destressed/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_newdic_destressed.json
to
	g = GD_AmE/AmE-diphones-IPA-annotated-columns.csv
Result saved to
	GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv
 
Creating notebook GD_AmE-diphones - LTR_newdic_destressed alignment application to LTR_newdic_destressed.ipynb w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_newdic_destressed/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_newdic_destressed.json
	LTR_newdic_destressed/LTR_newdic_destressed.tsv
	LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv


Input Notebook:  Align transcriptions.ipynb
Output Notebook: GD_AmE-diphones - LTR_newdic_destressed alignment application to LTR_newdic_destressed.ipynb
100%|██████████| 64/64 [00:02<00:00, 24.53it/s]


Finished applying alignment projection
	p = LTR_newdic_destressed_aligned_w_GD_AmE_destressed/alignment_of_LTR_newdic_destressed_w_AmE-diphones-IPA-annotated-columns.json
to
	l = LTR_newdic_destressed/LTR_newdic_destressed.tsv
Result saved to
	LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv


Creating notebook 'GD_AmE-diphones - LTR_CMU_destressed alignment application to AmE-diphones.ipynb' w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_CMU_destressed/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_CMU_destressed.json
	GD_AmE/AmE-diphones-IPA-annotated-columns.csv
	GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv


...

Finished applying alignment projection
	p = LTR_CMU_destressed_aligned_w_GD_AmE_destressed/alignment_of_LTR_CMU_destressed_w_AmE-diphones-IPA-annotated-columns.json
to
	l = LTR_CMU_destressed/LTR_CMU_destressed.tsv
Result saved to
	LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv


Creating notebook 'GD_AmE-diphones - LTR_Buckeye alignment application to AmE-diphones.ipynb' w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_Buckeye/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json
	GD_AmE/AmE-diphones-IPA-annotated-columns.csv
	GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv


Input Notebook:  Align transcriptions.ipynb
Output Notebook: GD_AmE-diphones - LTR_Buckeye alignment application to AmE-diphones.ipynb
100%|██████████| 64/64 [00:07<00:00,  8.25it/s]


Finished applying alignment projection
	p = GD_AmE_destressed_aligned_w_LTR_Buckeye/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json
to
	g = GD_AmE/AmE-diphones-IPA-annotated-columns.csv
Result saved to
	GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv
 
Creating notebook GD_AmE-diphones - LTR_Buckeye alignment application to LTR_Buckeye.ipynb w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_Buckeye/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json
	LTR_Buckeye/LTR_Buckeye.tsv
	LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv


Input Notebook:  Align transcriptions.ipynb
Output Notebook: GD_AmE-diphones - LTR_Buckeye alignment application to LTR_Buckeye.ipynb
100%|██████████| 64/64 [00:02<00:00, 27.12it/s]


Finished applying alignment projection
	p = LTR_Buckeye_aligned_w_GD_AmE_destressed/alignment_of_LTR_Buckeye_w_AmE-diphones-IPA-annotated-columns.json
to
	l = LTR_Buckeye/LTR_Buckeye.tsv
Result saved to
	LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv




# Step 2: Generating channel and (orthographic) lexicon distributions

## Step 2a: Generating channel distributions and associated metadata

In [19]:
%ls -d GD_*

 GD_AmE/
 GD_AmE_destressed_aligned_w_LTR_Buckeye/
 GD_AmE_destressed_aligned_w_LTR_CMU_destressed/
 GD_AmE_destressed_aligned_w_LTR_newdic_destressed/
'GD_AmE-diphones - LTR_Buckeye alignment application to AmE-diphones.ipynb'
'GD_AmE-diphones - LTR_Buckeye alignment application to LTR_Buckeye.ipynb'
'GD_AmE-diphones - LTR_Buckeye alignment definition.ipynb'
'GD_AmE-diphones - LTR_CMU_destressed alignment application to AmE-diphones.ipynb'
'GD_AmE-diphones - LTR_CMU_destressed alignment application to LTR_CMU_destressed.ipynb'
'GD_AmE-diphones - LTR_CMU_destressed alignment definition.ipynb'
'GD_AmE-diphones - LTR_newdic_destressed alignment application to AmE-diphones.ipynb'
'GD_AmE-diphones - LTR_newdic_destressed alignment application to LTR_newdic_destressed.ipynb'
'GD_AmE-diphones - LTR_newdic_destressed alignment definition.ipynb'


In [20]:
gating_data_folders = ('GD_AmE', ) + tuple(map(lambda ab: ab['gd_alignment_dn'], alignment_arg_bundles))
gating_data_folders

('GD_AmE',
 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed',
 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed',
 'GD_AmE_destressed_aligned_w_LTR_Buckeye')

In [21]:
gating_data_fps = ('GD_AmE/AmE-diphones-IPA-annotated-columns.csv',) + \
                  tuple(map(lambda ab: ab['my_og'], alignment_arg_bundles))
gating_data_fps

('GD_AmE/AmE-diphones-IPA-annotated-columns.csv',
 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv',
 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv',
 'GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv')

### Metadata

First (for downstream convenience) we identify the $n$-phones (not) contained in and (not) constructible from each of the versions of the gating data.

**NB:** This is done by calling the notebook `Run n-phone analysis of gating data.ipynb`, passing it **the filepath to a gating data `.csv`/`.tsv` file** and **a path to an output directory** for the dozen or so files the notebook will produce.

In [25]:
# takes ~2m on wittgenstein
for gating_data_fp in gating_data_fps:
    gd_dir = path.dirname(gating_data_fp)
    nb = pm.execute_notebook(
        'Run n-phone analysis of gating data.ipynb',
#         args['align_apply_ltr_nb_output_name'],
        path.join(gd_dir, gd_dir) + " n-phone analysis.ipynb",
        parameters=dict(g = gating_data_fp,
                        o = gd_dir)
    )
    listdir(gd_dir)
    print('\n')

Input Notebook:  Run n-phone analysis of gating data.ipynb
Output Notebook: GD_AmE/GD_AmE n-phone analysis.ipynb
100%|██████████| 43/43 [00:27<00:00,  1.57it/s]


['destressed response diphones.txt',
 'destressed stimuli diphone-based constructible triphones.txt',
 'stressed stimuli diphone-based constructible triphones.txt',
 'stressed stimuli uniphones.txt',
 'destressed stimuli uniphones.txt',
 'destressed stimuli illegal diphones.txt',
 'AmE-diphones-IPA-annotated-columns.csv',
 'destressed response illegal diphones.txt',
 'destressed response diphone-based constructible triphones.txt',
 'stressed stimuli illegal diphones.txt',
 'destressed stimuli diphone-based illegal triphones.txt',
 'destressed stimuli diphones.txt',
 'destressed response uniphones.txt',
 'stressed stimuli diphones.txt',
 'stressed stimuli diphone-based illegal triphones.txt',
 '.ipynb_checkpoints',
 'destressed response diphone-based illegal triphones.txt',
 'GD_AmE n-phone analysis.ipynb']

Input Notebook:  Run n-phone analysis of gating data.ipynb
Output Notebook: GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE_destressed_aligned_w_LTR_newdic_destressed n-phone analysis.ipynb
100%|██████████| 43/43 [00:27<00:00,  1.58it/s]


['destressed response diphone-based constructible triphones.txt',
 'destressed response diphones.txt',
 'alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_newdic_destressed.json',
 'GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv',
 'destressed response uniphones.txt',
 'destressed stimuli uniphones.txt',
 'stressed stimuli diphone-based illegal triphones.txt',
 'destressed stimuli diphone-based constructible triphones.txt',
 'destressed stimuli diphone-based illegal triphones.txt',
 'stressed stimuli diphone-based constructible triphones.txt',
 'destressed stimuli diphones.txt',
 'destressed response diphone-based illegal triphones.txt',
 'destressed stimuli illegal diphones.txt',
 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed n-phone analysis.ipynb',
 'stressed stimuli illegal diphones.txt',
 'stressed stimuli uniphones.txt',
 'destressed response illegal diphones.txt',
 'stressed stimuli diphones.txt']

Input Notebook:  Run n-phone analysis of gating data.ipynb
Output Notebook: GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE_destressed_aligned_w_LTR_CMU_destressed n-phone analysis.ipynb
100%|██████████| 43/43 [00:26<00:00,  1.63it/s]


['destressed response diphone-based constructible triphones.txt',
 'GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv',
 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed n-phone analysis.ipynb',
 'destressed response diphone-based illegal triphones.txt',
 'alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_CMU_destressed.json',
 'destressed stimuli diphone-based constructible triphones.txt',
 'destressed response uniphones.txt',
 'destressed response diphones.txt',
 'stressed stimuli diphone-based illegal triphones.txt',
 'destressed stimuli diphone-based illegal triphones.txt',
 'destressed stimuli uniphones.txt',
 'stressed stimuli uniphones.txt',
 'destressed stimuli diphones.txt',
 'stressed stimuli illegal diphones.txt',
 'stressed stimuli diphones.txt',
 'destressed response illegal diphones.txt',
 'destressed stimuli illegal diphones.txt',
 'stressed stimuli diphone-based constructible triphones.txt']

Input Notebook:  Run n-phone analysis of gating data.ipynb
Output Notebook: GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE_destressed_aligned_w_LTR_Buckeye n-phone analysis.ipynb
100%|██████████| 43/43 [00:26<00:00,  1.59it/s]


['GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv',
 'stressed stimuli uniphones.txt',
 'destressed stimuli diphone-based constructible triphones.txt',
 'stressed stimuli diphone-based constructible triphones.txt',
 'GD_AmE_destressed_aligned_w_LTR_Buckeye n-phone analysis.ipynb',
 'destressed stimuli diphone-based illegal triphones.txt',
 'stressed stimuli diphones.txt',
 'destressed response diphone-based constructible triphones.txt',
 'destressed stimuli illegal diphones.txt',
 'alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json',
 'destressed response diphone-based illegal triphones.txt',
 'destressed response uniphones.txt',
 'stressed stimuli diphone-based illegal triphones.txt',
 'stressed stimuli illegal diphones.txt',
 'destressed stimuli uniphones.txt',
 'destressed stimuli diphones.txt',
 'destressed response illegal diphones.txt',
 'destressed response diphones.txt']

### Channel distributions

Next, the notebook `Producing channel distributions.ipynb` will create `.json` files defining (among other things) a uniphone and triphone channel distribution. It requires the following arguments to specify information about what kind of channel model to build and where to put it:
 - **a filepath** to a gating data file
 - **a directory** containing metadata indicating possible/impossible $n$-phones
 - **a string argument** ("stressed" or "destressed") indicating whether the distribution will be over a segment inventory with or without stress information
 - **a real valued**  smoothing parameter (a pseudocount to add to every channel outcome)
 - **an output directory** to write the channel model to.

In [22]:
pseudocounts = (0, 0.01, 0.05, 0.1)

In [23]:
cm_arg_bundles = []
for gating_data_fp in gating_data_fps:
    metadata_dir = path.dirname(gating_data_fp)
    s = "destressed"
    channel_model_dir_stem = 'CM' + metadata_dir[2:]
    
    for pc in pseudocounts:
        channel_model_dir_suffix = '_pseudocount' + str(pc)
        if metadata_dir == 'GD_AmE':
            channel_model_dir = channel_model_dir_stem + '_' + s + '_unaligned' + channel_model_dir_suffix
        else:
            channel_model_dir = channel_model_dir_stem + channel_model_dir_suffix
        nb_output_name = 'Producing channel distributions from ' + metadata_dir + ', pc={0}'.format(pc) + '.ipynb'
        new_arg_bundle = {'gating_data_fp':gating_data_fp,
                          'metadata_dir':metadata_dir,
                          's':s,
                          'c':pc,
                          'cm_dir':channel_model_dir,
                          'nb_output_name':nb_output_name}
        cm_arg_bundles.append(new_arg_bundle)

In [24]:
cm_arg_bundles[0]

{'gating_data_fp': 'GD_AmE/AmE-diphones-IPA-annotated-columns.csv',
 'metadata_dir': 'GD_AmE',
 's': 'destressed',
 'c': 0,
 'cm_dir': 'CM_AmE_destressed_unaligned_pseudocount0',
 'nb_output_name': 'Producing channel distributions from GD_AmE, pc=0.ipynb'}

In [45]:
# takes ~110m on wittgenstein
for ab in cm_arg_bundles:
    ensure_dir_exists(ab['cm_dir'])
    
    nb = pm.execute_notebook(
    'Producing channel distributions.ipynb',
    path.join(ab['cm_dir'], ab['nb_output_name']),
    parameters=dict(g = ab['gating_data_fp'],
                    m = ab['metadata_dir'],
                    s = ab['s'],
                    c = ab['c'],
                    o = ab['cm_dir'])
    )
    print('\n')

Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_unaligned_pseudocount0/Producing channel distributions from GD_AmE, pc=0.ipynb
100%|██████████| 189/189 [03:01<00:00,  2.88it/s]


Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_unaligned_pseudocount0.01/Producing channel distributions from GD_AmE, pc=0.01.ipynb
100%|██████████| 189/189 [07:47<00:00,  2.69s/it]


Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_unaligned_pseudocount0.05/Producing channel distributions from GD_AmE, pc=0.05.ipynb
100%|██████████| 189/189 [08:01<00:00,  2.59s/it]


Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_unaligned_pseudocount0.1/Producing channel distributions from GD_AmE, pc=0.1.ipynb
100%|██████████| 189/189 [07:48<00:00,  2.65s/it]


Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0/Producing channel distributions from GD_AmE_destressed_aligned_w_LTR_newdic_destressed, pc=0.ipynb
100%|██████████| 189/189 [03:00<00:00,  1.04it/s]


...

Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/Producing channel distributions from GD_AmE_destressed_aligned_w_LTR_CMU_destressed, pc=0.1.ipynb
100%|██████████| 189/189 [07:47<00:00,  2.65s/it]


Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0/Producing channel distributions from GD_AmE_destressed_aligned_w_LTR_Buckeye, pc=0.ipynb
100%|██████████| 189/189 [02:59<00:00,  1.05it/s]


Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/Producing channel distributions from GD_AmE_destressed_aligned_w_LTR_Buckeye, pc=0.01.ipynb
100%|██████████| 189/189 [07:49<00:00,  2.65s/it]


Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Producing channel distributions from GD_AmE_destressed_aligned_w_LTR_Buckeye, pc=0.05.ipynb
100%|██████████| 189/189 [07:57<00:00,  3.62s/it]


Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/Producing channel distributions from GD_AmE_destressed_aligned_w_LTR_Buckeye, pc=0.1.ipynb
100%|██████████| 189/189 [07:55<00:00,  3.67s/it]


**How are channel distributions created?**

Take a look at `Producing channel distributions.ipynb`. Besides removing stress information, there are some mathematically non-trivial details that go into defining both uniphone and triphone channel distributions.

## Step 2b: Generating (contextual) lexicon distributions (over orthographic vocabularies)

Given 
 - a language model $m$ (defined by a `.arpa` file or `kenlm` memory-mapped analogue) 
 - a choice of n-gram contexts $C$ (a `.txt` file with one context per line)
 - a vocabulary $V$ (a `.txt` file with one word per line)
 - a (partial) output filepath $o$ / output filepath prefix $o$
 
`Producing contextual distributions.ipynb` will write a serialized/memory-mapped `numpy` array to $o$.hV_C that defines $-log_2( p(V|C) )$ - slightly transformed output from `kenlm`. It will also write $p(V|C)$ to $o$.pV_C, and copy both $V$ and $C$ to the base directory specified by $o$. (In both cases, each column is associated with the distribution $p(V|c)$ for some $c$.)

In [25]:
contexts
fisher_lm_fps

('buckeye_contexts.txt', 'swbd2003_contexts.txt')

{'lm': 'LM_Fisher/fisher_utterances_main_4gram.mmap',
 'vocab': 'LM_Fisher/fisher_vocabulary_main.txt'}

In [26]:
LDs = [{'LD_dir':'LD_Fisher_vocab_in_Buckeye_contexts',
        'o_fn_stem':'LD_fisher_vocab_in_buckeye_contexts',
        'o':'LD_Fisher_vocab_in_Buckeye_contexts' + '/' + 'LD_fisher_vocab_in_buckeye_contexts',
        'm':fisher_lm_fps['lm'],
        'v':fisher_lm_fps['vocab'],
        'c':buckeye_contexts,
        'nb_fp':path.join('LD_Fisher_vocab_in_Buckeye_contexts', 
                          'Producing ' + 'LD_Fisher_vocab_in_Buckeye_contexts'.replace('_', ' ')[3:] + ' contextual distributions.ipynb')},
       {'LD_dir':'LD_Fisher_vocab_in_swbd2003_contexts',
        'o_fn_stem':'LD_fisher_vocab_in_swbd2003_contexts',
        'o':'LD_Fisher_vocab_in_swbd2003_contexts' + '/' + 'LD_fisher_vocab_in_swbd2003_contexts',
        'm':fisher_lm_fps['lm'],
        'v':fisher_lm_fps['vocab'],
        'c':swbd2003_contexts,
        'nb_fp':path.join('LD_Fisher_vocab_in_swbd2003_contexts', 
                          'Producing ' + 'LD_Fisher_vocab_in_swbd2003_contexts'.replace('_', ' ')[3:] + ' contextual distributions.ipynb')}
      ]

In [248]:
# takes ~80m on wittgenstein
for ld in LDs:
    output_dir = path.dirname(ld['LD_dir'])
    ensure_dir_exists(output_dir)
    
    nb = pm.execute_notebook(
    'Producing contextual distributions.ipynb',
#     'Producing ' + ld['LD_dir'].replace('_', ' ')[3:] + ' contextual distributions.ipynb',
    ld['nb_fp'],
    parameters=dict(m = ld['m'],
                    v = ld['v'],
                    c = ld['c'],
                    o = ld['o'])
    )
    print('\n')

Input Notebook:  Producing contextual distributions.ipynb
Output Notebook: LD_Fisher_vocab_in_Buckeye_contexts/Producing Fisher vocab in Buckeye contexts contextual distributions.ipynb










  0%|          | 0/86 [00:00<?, ?it/s]









  3%|▎         | 3/86 [00:00<00:03, 24.86it/s]









  7%|▋         | 6/86 [00:00<00:03, 26.09it/s]









 10%|█         | 9/86 [00:00<00:02, 26.96it/s]









 14%|█▍        | 12/86 [00:00<00:02, 26.84it/s]









 17%|█▋        | 15/86 [00:00<00:02, 26.56it/s]









 21%|██        | 18/86 [00:00<00:02, 25.03it/s]









 24%|██▍       | 21/86 [00:00<00:03, 18.35it/s]









 28%|██▊       | 24/86 [00:01<00:03, 19.58it/s]









 31%|███▏      | 27/86 [00:01<00:02, 20.06it/s]









 35%|███▍      | 30/86 [00:01<00:02, 19.89it/s]









 38%|███▊      | 33/86 [00:01<00:02, 21.35it/s]









 42%|████▏     | 36/86 [00:01<00:02, 21.42it/s]









 45%|████▌     | 39/86 [00:01<00:02, 21.26it/s]









 49%|████▉     | 42

Input Notebook:  Producing contextual distributions.ipynb
Output Notebook: LD_Fisher_vocab_in_swbd2003_contexts/Producing Fisher vocab in swbd2003 contexts contextual distributions.ipynb










  0%|          | 0/87 [00:00<?, ?it/s]









  3%|▎         | 3/87 [00:00<00:03, 24.85it/s]









  8%|▊         | 7/87 [00:00<00:03, 26.28it/s]









 11%|█▏        | 10/87 [00:00<00:02, 26.92it/s]









 15%|█▍        | 13/87 [00:00<00:02, 26.42it/s]









 18%|█▊        | 16/87 [00:00<00:03, 21.61it/s]









 22%|██▏       | 19/87 [00:00<00:03, 22.28it/s]









 25%|██▌       | 22/87 [00:01<00:03, 17.31it/s]









 29%|██▊       | 25/87 [00:01<00:03, 19.44it/s]









 32%|███▏      | 28/87 [00:01<00:03, 17.38it/s]









 34%|███▍      | 30/87 [00:01<00:03, 17.08it/s]









 38%|███▊      | 33/87 [00:01<00:02, 18.75it/s]









 41%|████▏     | 36/87 [00:01<00:02, 19.88it/s]









 45%|████▍     | 39/87 [00:01<00:02, 21.15it/s]









 48%|████▊     |

# Step 3: Creating combinable models

**The basic problem:**

1. **Channel model + transcribed lexicon relation**: Even after the gating data and a transcribed lexicon relation are defined over the same inventory, 
 - the lexicon may contain triphones that are not in the stimuli triphones of a triphone channel model.
 - the channel distribution will contain triphones that cannot be found in the transcribed lexicon relation. (While the other steps here are strictly necessary, this is simply a practical step for making downstream computation faster.)
2. **Language model + transcribed lexicon relation**: The orthographic vocabulary of a transcribed lexicon relation may contain wordforms not in an n-gram model's vocabulary. (We *don't* want to use the out-of-vocabulary estimate for those wordforms.)
3.  **Contextual distributions + transcribed lexicon relation**: The contextual distributions from Step 3b above are defined over the *language model's* orthographic vocabulary, which will likely include wordforms that are not in the transcribed lexicon relation. We want to create modified forms of these distributions where we condition on the choice of an orthographic wordform that is in the transcribed lexicon relation.

Once we have
 - a version $l'$ of the transcribed lexicon relation $l$ trimmed with respect to both the triphones of the channel model $c$ and the (orthographic) vocabulary of a language model $m$
 - a version $d'$ of the contextual distributions over $m$'s vocabulary (with respect to some set of n-gram contexts) $d$ trimmed to only define distributions over $l'$
 - a version $c'$ of the channel model $c$ trimmed with respect to a transcribed lexicon relation $l'$
 - a probability distribution over segmental wordforms given an orthographic wordform
 
we will be able to combine everything together to calculate confusability of wordforms in corpus contexts.

## Step 3a: Filter transcription lexicons to only include words that can be modeled by a given channel distribution

In [ ]:
#gather relevant LTRs and their associated CMs

In [27]:
aligned_LTRs = list(map(lambda ab: {'LTR_fp':ab['my_ol'],
                                    'GD_fp':ab['my_og']},
                        alignment_arg_bundles))
list(map(lambda d: d['LTR_fp'],
         aligned_LTRs))

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv']

In [28]:
# CM_dirs = list(map(lambda cab: {'CM_fp':path.join(cab['cm_dir'],'pY1X0X1X2.json'),
#                                 'GD_fp':cab['gating_data_fp']},
#                    filter(lambda cab: cab['c'] != 0, cm_arg_bundles)))
# CM_dirs
# CM_dirs[5]
# # listdir(CM_dirs[0][])

aligned_CMs = list(map(lambda cab: {'CM_fp':path.join(cab['cm_dir'],'pY1X0X1X2.json'),
                                    'GD_fp':cab['gating_data_fp']},
                       filter(lambda cab: cab['c'] != 0 and 'aligned' in cab['gating_data_fp'], 
                              cm_arg_bundles)))
aligned_CMs

[{'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv'},
 {'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv'},
 {'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv'},
 {'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv'},
 {'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diph

In [29]:
def get_aligned_CMs(ltr_bundle):
    matches = [cm_bundle for cm_bundle in aligned_CMs if cm_bundle['GD_fp'] == ltr_bundle['GD_fp']]
    return list(map(lambda d: d['CM_fp'],
                    matches))

In [30]:
aligned_LTRs[0]['LTR_fp']

#NB: all of these will have the same set of stimuli triphones
#    ...which is all we care about here
get_aligned_CMs(aligned_LTRs[0]) 

'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv'

['CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json']

In [31]:
aligned_LTRs_and_CM = [{'LTR_fp':ltr['LTR_fp'],
                        'matching_CMs':get_aligned_CMs(ltr)}
                       for ltr in aligned_LTRs]
aligned_LTRs_and_CM

[{'LTR_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv',
  'matching_CMs': ['CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json']},
 {'LTR_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv',
  'matching_CMs': ['CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/pY1X0X1X2.json']},
 {'LTR_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv',
  'matching_CMs': ['CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR

In [32]:
for each in aligned_LTRs_and_CM:
    each['c'] = each['matching_CMs'][0]
    each['l'] = each['LTR_fp']
    o_dir = path.dirname(each['LTR_fp'])
    o_fn = path.basename(each['LTR_fp']).split('w_')[0] + 'CM_filtered' + '.tsv'
    each['o'] = path.join(o_dir, o_fn)
    
    nb_fn = 'Filter ' + o_fn.split('_aligned')[0] + ' against channel model.ipynb'
    each['nb_fp'] = path.join(o_dir, nb_fn)
    
    print('c = {0}\nl = {1}\no = {2}\nnb = {3}'.format(each['c'], each['l'], each['o'], each['nb_fp']))
    print(' ')

c = CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json
l = LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv
o = LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
nb = LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Filter LTR_newdic_destressed against channel model.ipynb
 
c = CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json
l = LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv
o = LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
nb = LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Filter LTR_CMU_destressed against channel model.ipynb
 
c = CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/pY1X0X1X2.json
l = LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv
o = LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Bu

In [38]:
# takes about 20s on wittgenstein
for each in aligned_LTRs_and_CM:
    output_dir = path.dirname(each['o'])
    ensure_dir_exists(output_dir)
    
    nb = pm.execute_notebook(
    'Filter transcription lexicon by channel model.ipynb',
    each['nb_fp'],
    parameters=dict(l = each['l'],
                    c = each['c'],
                    o = each['o'])
    )
    print('\n')

Input Notebook:  Filter transcription lexicon by channel model.ipynb
Output Notebook: LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Filter LTR_newdic_destressed against channel model.ipynb
100%|██████████| 27/27 [00:02<00:00,  8.01it/s]


Input Notebook:  Filter transcription lexicon by channel model.ipynb
Output Notebook: LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Filter LTR_CMU_destressed against channel model.ipynb
100%|██████████| 27/27 [00:04<00:00,  4.22it/s]


Input Notebook:  Filter transcription lexicon by channel model.ipynb
Output Notebook: LTR_Buckeye_aligned_w_GD_AmE_destressed/Filter LTR_Buckeye against channel model.ipynb
100%|██████████| 27/27 [00:02<00:00, 11.92it/s]


## Step 3b: Filter transcription lexicons to only include words that are in a language model's vocabulary

**NB** For this step, we will use shell commands provided by the `csvkit` package.

In [33]:
fisher_lm_fps
fisher_lm_vocab_fp

{'lm': 'LM_Fisher/fisher_utterances_main_4gram.mmap',
 'vocab': 'LM_Fisher/fisher_vocabulary_main.txt'}

'LM_Fisher/fisher_vocabulary_main.txt'

In [34]:
LTR_fps = list(map(lambda pair: pair['LTR_fp'],
                   aligned_LTRs))
LTR_fps

LTR_CM_filtered = list(map(lambda d: d['o'],
                           aligned_LTRs_and_CM))
LTR_CM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv']

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv']

How many lexicon entries have unmodelable triphones? (We'll next check how many such lexicon entries aren't in the language model vocabulary.)

In [35]:
!wc -l LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv
!wc -l LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv
!wc -l LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv

19529 LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv
133855 LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv
7999 LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv


In [36]:
!wc -l LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
!wc -l LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
!wc -l LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv

17079 LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
127799 LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
7011 LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv


Word type loss for Buckeye:
 - $7999 - 7011 = 988$ word types lost due to unmodelable triphones

In [37]:
LTR_CM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv']

In [38]:
LTR_LM_filter_bundles = []
for each_LTR_fp in LTR_CM_filtered:
    bundle = dict()
    LTR_descr = path.basename(each_LTR_fp)[:-4]
    LM_V_descr = path.basename(fisher_lm_vocab_fp)[:-4]
    bundle['l'] = each_LTR_fp
    bundle['v'] = fisher_lm_vocab_fp
    bundle['o'] = each_LTR_fp[:-4] + '_LM_filtered' + '.tsv'
    bundle['nb_fp'] = path.join(path.dirname(each_LTR_fp),
                                f'Filter {LTR_descr} against {LM_V_descr}' + '.ipynb')
    bundle
    LTR_LM_filter_bundles.append(bundle)
    print('')

{'l': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv',
 'v': 'LM_Fisher/fisher_vocabulary_main.txt',
 'o': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'nb_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Filter LTR_newdic_destressed_aligned_CM_filtered against fisher_vocabulary_main.ipynb'}

{'l': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv',
 'v': 'LM_Fisher/fisher_vocabulary_main.txt',
 'o': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'nb_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Filter LTR_CMU_destressed_aligned_CM_filtered against fisher_vocabulary_main.ipynb'}

{'l': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv',
 'v': 'LM_Fisher/fisher_vocabulary_main.txt',
 'o': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
 'nb_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/Filter LTR_Buckeye_aligned_CM_filtered against fisher_vocabulary_main.ipynb'}

In [45]:
# takes about ~1m on wittgenstein
for each in LTR_LM_filter_bundles:
    output_dir = path.dirname(each['o'])
    ensure_dir_exists(output_dir)
    
    
    nb = pm.execute_notebook(
    'Filter transcription lexicon by language model vocabulary.ipynb',
    each['nb_fp'],
    parameters=dict(l = each['l'],
                    v = each['v'],
                    o = each['o'])
    )
    print('\n')

Input Notebook:  Filter transcription lexicon by language model vocabulary.ipynb
Output Notebook: LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Filter LTR_newdic_destressed_aligned_CM_filtered against fisher_vocabulary_main.ipynb
100%|██████████| 25/25 [00:05<00:00,  4.65it/s]


Input Notebook:  Filter transcription lexicon by language model vocabulary.ipynb
Output Notebook: LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Filter LTR_CMU_destressed_aligned_CM_filtered against fisher_vocabulary_main.ipynb
100%|██████████| 25/25 [00:44<00:00,  1.78s/it]


Input Notebook:  Filter transcription lexicon by language model vocabulary.ipynb
Output Notebook: LTR_Buckeye_aligned_w_GD_AmE_destressed/Filter LTR_Buckeye_aligned_CM_filtered against fisher_vocabulary_main.ipynb
100%|██████████| 25/25 [00:02<00:00,  7.92it/s]


**A bit less than half** of the triphone channel model-modelable `newdic` lexicon **isn't** in the LM vocab:

In [39]:
!wc -l LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
!wc -l LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv

17079 LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
9412 LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv


**About three quarters** of the triphone channel model-modelable `CMU_destressed` lexicon **isn't** in the LM vocab:

In [40]:
!wc -l LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
!wc -l LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv

127799 LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
33125 LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv


**Less than 10%** of the triphone channel model-modelable `Buckeye` lexicon **isn't** in the LM vocab:

In [41]:
!wc -l LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv
!wc -l LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv

7011 LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv
6575 LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv


Word type loss for Buckeye:
 - $7999 - 7011 = 988$ word types lost due to unmodelable triphones
 - $7011 - 6575 = 436$ further word types lost due orthographic wordforms not being in the language model vocabulary
 - $\frac{436+988}{7999} ≈ 17.8\%$ of word types lost.

Collecting the filtered transcribed lexicon relations...

In [42]:
LTR_CM_filtered_LM_filtered = list(map(lambda bundle: bundle['o'],
                                       LTR_LM_filter_bundles))
LTR_CM_filtered_LM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv']

## Step 3c: Filter the conditioning events of channel distributions to only include triphones contained in a transcription lexicon's segmental wordform list

In [43]:
#what channel models might you want to use with what lexicons?
# there are 3x3 triphone CMs that are aligned with one of 3 LTRs and have one of 3 relevant pseudocount levels
# For each of the 3 LTRs aligned with the gating data, there's exactly 1 `LTR...CM_filtered_LM_filtered.tsv` file
# ∴ there are 3x3 channel models to trim

In [44]:
for each in aligned_LTRs_and_CM:
    LTR_dir = path.dirname(each['LTR_fp'])
    LTR_trimmed_fn = path.basename(each['LTR_fp']).split('_w_')[0] + '_CM_filtered_LM_filtered.tsv'
    each['LTR_trimmed_fp'] = path.join(LTR_dir, LTR_trimmed_fn)
    each['matching_trimmed_CMs'] = [path.join(path.dirname(fp),
                                              LTR_trimmed_fn[:-4] + '_' + path.basename(fp))
                                    for fp in each['matching_CMs']]
    
#     each['LTR_fp']
    each['LTR_trimmed_fp']
    each['matching_CMs']
    each['matching_trimmed_CMs']
#     each['trimmed LTR_fp']
    print('')

'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv'

['CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json']

['CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json']

'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv'

['CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/pY1X0X1X2.json']

['CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json']

'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv'

['CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/pY1X0X1X2.json']

['CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json']

In [45]:
trimmed_LTR_CM_triples = []
for each in aligned_LTRs_and_CM:
    assert len(each['matching_CMs']) == len(each['matching_trimmed_CMs'])
    
    for i in range(len(each['matching_CMs'])):
        args = dict()
        args['l'] = each['LTR_trimmed_fp']
        args['c'] = each['matching_CMs'][i]
        args['o'] = each['matching_trimmed_CMs'][i]
        args['nb_fp'] = path.join(path.dirname(args['c']),
                                  f"Filter {path.dirname(args['c'])} against {path.basename(args['l'])[:-4]}.ipynb")
        args
        trimmed_LTR_CM_triples.append(args)
        print('')


{'l': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/Filter CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01 against LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Filter CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05 against LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Filter CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1 against LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/Filter CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01 against LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Filter CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05 against LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/Filter CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1 against LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/Filter CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01 against LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Filter CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05 against LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/Filter CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1 against LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}

In [52]:
# takes about 70s on wittgenstein
for each in trimmed_LTR_CM_triples:
    output_dir = path.dirname(each['o'])
    ensure_dir_exists(output_dir)
#     if not path.exists(output_dir):
#         print(f"Creating output path '{output_dir}'")
#         makedirs(output_dir)
    
    nb = pm.execute_notebook(
    'Filter channel model by transcription lexicon.ipynb',
    each['nb_fp'],
    parameters=dict(l = each['l'],
                    c = each['c'],
                    o = each['o'])
    )
    print('\n')

Input Notebook:  Filter channel model by transcription lexicon.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/Filter CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01 against LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 32/32 [00:04<00:00,  7.78it/s]


Input Notebook:  Filter channel model by transcription lexicon.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Filter CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05 against LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 32/32 [00:04<00:00,  6.79it/s]


Input Notebook:  Filter channel model by transcription lexicon.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Filter CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1 against LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 32/32 [00:04<00:00,  6.70it/s]


Input Notebook:  Filter channel model by transcription lexicon.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/Filter CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01 against LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 32/32 [00:08<00:00,  3.83it/s]


Input Notebook:  Filter channel model by transcription lexicon.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Filter CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05 against LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 32/32 [00:08<00:00,  3.60it/s]


Input Notebook:  Filter channel model by transcription lexicon.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/Filter CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1 against LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 32/32 [00:09<00:00,  3.41it/s]


Input Notebook:  Filter channel model by transcription lexicon.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/Filter CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01 against LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 32/32 [00:03<00:00,  9.41it/s]


Input Notebook:  Filter channel model by transcription lexicon.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Filter CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05 against LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 32/32 [00:04<00:00,  7.97it/s]


Input Notebook:  Filter channel model by transcription lexicon.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/Filter CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1 against LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 32/32 [00:04<00:00,  7.57it/s]


## Step 3d: For each (filtered) transcribed lexicon relation, define the relevant contextual lexicon distributions over orthographic wordforms

In [46]:
LTR_CM_filtered_LM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv']

In [47]:
LDs

[{'LD_dir': 'LD_Fisher_vocab_in_Buckeye_contexts',
  'o_fn_stem': 'LD_fisher_vocab_in_buckeye_contexts',
  'o': 'LD_Fisher_vocab_in_Buckeye_contexts/LD_fisher_vocab_in_buckeye_contexts',
  'm': 'LM_Fisher/fisher_utterances_main_4gram.mmap',
  'v': 'LM_Fisher/fisher_vocabulary_main.txt',
  'c': 'buckeye_contexts.txt',
  'nb_fp': 'LD_Fisher_vocab_in_Buckeye_contexts/Producing Fisher vocab in Buckeye contexts contextual distributions.ipynb'},
 {'LD_dir': 'LD_Fisher_vocab_in_swbd2003_contexts',
  'o_fn_stem': 'LD_fisher_vocab_in_swbd2003_contexts',
  'o': 'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts',
  'm': 'LM_Fisher/fisher_utterances_main_4gram.mmap',
  'v': 'LM_Fisher/fisher_vocabulary_main.txt',
  'c': 'swbd2003_contexts.txt',
  'nb_fp': 'LD_Fisher_vocab_in_swbd2003_contexts/Producing Fisher vocab in swbd2003 contexts contextual distributions.ipynb'}]

In [48]:
fisher_lm_vocab_fp

'LM_Fisher/fisher_vocabulary_main.txt'

In [49]:
buckeye_contexts
swbd2003_contexts

'buckeye_contexts.txt'

'swbd2003_contexts.txt'

In [50]:
LD_projection_args = [
    {'d':'LD_Fisher_vocab_in_Buckeye_contexts/LD_fisher_vocab_in_buckeye_contexts.pV_C',
     'v':fisher_lm_vocab_fp,
     'c':buckeye_contexts,
     'l':'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
     'o':'LD_Fisher_vocab_in_Buckeye_contexts/LD_fisher_vocab_in_buckeye_contexts' + '_projected_' + 'LTR_Buckeye' + '.pV_C',
     'f':'True',
     'nb_fp':path.join('LD_Fisher_vocab_in_Buckeye_contexts',
                       'Filter ' + 'LD_fisher_vocab_in_buckeye_contexts' + ' against ' + 'LTR_Buckeye_aligned_CM_filtered_LM_filtered' + '.ipynb')},
    {'d':'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts.pV_C',
     'v':fisher_lm_vocab_fp,
     'c':swbd2003_contexts,
     'l':'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
     'o':'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts' + '_projected_' + 'LTR_CMU_destressed' + '.pV_C',
     'f':'True',
     'nb_fp':path.join('LD_Fisher_vocab_in_swbd2003_contexts',
                       'Filter ' + 'LD_fisher_vocab_in_swbd2003_contexts' + ' against ' + 'LTR_CMU_destressed_aligned_CM_filtered_LM_filtered' + '.ipynb')},
    {'d':'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts.pV_C',
     'v':fisher_lm_vocab_fp,
     'c':swbd2003_contexts,
     'l':'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
     'o':'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts' + '_projected_' + 'LTR_newdic_destressed' + '.pV_C',
     'f':'True',
     'nb_fp':path.join('LD_Fisher_vocab_in_swbd2003_contexts',
                       'Filter ' + 'LD_fisher_vocab_in_swbd2003_contexts' + ' against ' + 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered' + '.ipynb')}
]


In [101]:
# takes about ~10-15m on wittgenstein:
#  ≤30s for Buckeye vocab + Buckeye contexts
#  ≈6.5-7m for CMU_destressed vocab in swbd2003 contexts
#  ≈3m for newdic_destressed vocab in swbd2003 contexts
for each in LD_projection_args:
    output_dir = path.dirname(each['o'])
    ensure_dir_exists(output_dir)
    
    nb = pm.execute_notebook(
    'Filter contextual lexicon distribution by transcription lexicon.ipynb',
    each['nb_fp'],
    parameters=dict(d = each['d'],
                    v = each['v'],
                    c = each['c'],
                    l = each['l'],
                    o = each['o'],
                    f = each['f'])
    )
    print('\n')

Input Notebook:  Filter contextual lexicon distribution by transcription lexicon.ipynb
Output Notebook: LD_Fisher_vocab_in_Buckeye_contexts/Filter LD_fisher_vocab_in_buckeye_contexts against LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb


  0%|          | 0/42 [00:00<?, ?it/s]

 12%|█▏        | 5/42 [00:00<00:00, 44.91it/s]

 26%|██▌       | 11/42 [00:00<00:00, 48.24it/s]

 33%|███▎      | 14/42 [00:00<00:01, 27.05it/s]

 45%|████▌     | 19/42 [00:00<00:00, 30.31it/s]

 52%|█████▏    | 22/42 [00:00<00:00, 26.78it/s]

 60%|█████▉    | 25/42 [00:02<00:02,  6.40it/s]

 64%|██████▍   | 27/42 [00:03<00:04,  3.40it/s]

 69%|██████▉   | 29/42 [00:11<00:18,  1.44s/it]

 76%|███████▌  | 32/42 [00:12<00:11,  1.13s/it]

 79%|███████▊  | 33/42 [00:14<00:12,  1.37s/it]

 83%|████████▎ | 35/42 [00:14<00:06,  1.02it/s]

 86%|████████▌ | 36/42 [00:15<00:04,  1.30it/s]

 90%|█████████ | 38/42 [00:15<00:02,  1.80it/s]

 95%|█████████▌| 40/42 [00:23<00:03,  1.57s/it]

100%|██████████| 42/42 [00:23<00

Input Notebook:  Filter contextual lexicon distribution by transcription lexicon.ipynb
Output Notebook: LD_Fisher_vocab_in_swbd2003_contexts/Filter LD_fisher_vocab_in_swbd2003_contexts against LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb


  0%|          | 0/42 [00:00<?, ?it/s]

 12%|█▏        | 5/42 [00:00<00:00, 49.02it/s]

 29%|██▊       | 12/42 [00:00<00:00, 39.60it/s]

 38%|███▊      | 16/42 [00:00<00:00, 39.36it/s]

 48%|████▊     | 20/42 [00:00<00:00, 36.97it/s]

 55%|█████▍    | 23/42 [00:00<00:00, 26.03it/s]

 55%|█████▍    | 23/42 [00:10<00:00, 26.03it/s]

 62%|██████▏   | 26/42 [00:12<00:19,  1.24s/it]

 69%|██████▉   | 29/42 [01:09<01:24,  6.51s/it]

 76%|███████▌  | 32/42 [01:15<00:51,  5.18s/it]

 79%|███████▊  | 33/42 [02:28<03:48, 25.42s/it]

 81%|████████  | 34/42 [02:30<02:28, 18.61s/it]

 86%|████████▌ | 36/42 [02:39<01:25, 14.30s/it]

 88%|████████▊ | 37/42 [02:42<00:54, 10.82s/it]

 93%|█████████▎| 39/42 [06:44<02:11, 43.97s/it]

 98%|█████████▊| 41/42 

Input Notebook:  Filter contextual lexicon distribution by transcription lexicon.ipynb
Output Notebook: LD_Fisher_vocab_in_swbd2003_contexts/Filter LD_fisher_vocab_in_swbd2003_contexts against LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb


  0%|          | 0/42 [00:00<?, ?it/s]

 14%|█▍        | 6/42 [00:00<00:00, 56.80it/s]

 29%|██▊       | 12/42 [00:00<00:01, 20.85it/s]

 40%|████      | 17/42 [00:00<00:01, 24.77it/s]

 48%|████▊     | 20/42 [00:01<00:00, 25.88it/s]

 55%|█████▍    | 23/42 [00:01<00:00, 26.68it/s]

 62%|██████▏   | 26/42 [00:04<00:05,  2.70it/s]

 62%|██████▏   | 26/42 [00:20<00:05,  2.70it/s]

 69%|██████▉   | 29/42 [01:01<01:16,  5.90s/it]

 76%|███████▌  | 32/42 [01:02<00:43,  4.31s/it]

 79%|███████▊  | 33/42 [01:57<02:54, 19.37s/it]

 81%|████████  | 34/42 [01:58<01:50, 13.80s/it]

 86%|████████▌ | 36/42 [02:00<01:00, 10.04s/it]

 88%|████████▊ | 37/42 [02:01<00:36,  7.26s/it]

 93%|█████████▎| 39/42 [03:12<00:47, 15.74s/it]

100%|██████████| 42/

## Step 3e: For each (filtered) transcribed lexicon relation, define a conditional distribution on segmental wordforms given an orthographic wordform

In [51]:
LTR_CM_filtered_LM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv']

In [52]:
pW_V_fp_bundles = []
for ltr_fp in LTR_CM_filtered_LM_filtered:
    LTR_n = path.basename( ltr_fp )[:-4]
    LTR_dir = path.dirname( ltr_fp )
    pW_V_fp_bundles.append({'LTR_fp':ltr_fp,
                          'pW_V_fp':ltr_fp[:-4],# + '.pW_V',
                          'nb_fp':path.join(LTR_dir,'Define pW_V given {0}.ipynb'.format(LTR_n))})
pW_V_fp_bundles

[{'LTR_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered',
  'nb_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Define pW_V given LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'},
 {'LTR_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered',
  'nb_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Define pW_V given LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'},
 {'LTR_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered',
  'nb_fp': 'LTR_Buckeye_aligned_w_GD_AmE_de

In [60]:
# takes about ~30s on wittgenstein
for each in pW_V_fp_bundles:
    pW_V_fp_output_dir = path.dirname(each['pW_V_fp'])
    ensure_dir_exists(pW_V_fp_output_dir)
    
    nb = pm.execute_notebook(
    'Define a conditional distribution on segmental wordforms given an orthographic one.ipynb',
    each['nb_fp'],
    parameters=dict(l = each['LTR_fp'],
                    o = each['pW_V_fp'])
    )
    print('\n')

Input Notebook:  Define a conditional distribution on segmental wordforms given an orthographic one.ipynb
Output Notebook: LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Define pW_V given LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 61/61 [00:07<00:00,  8.20it/s]


Input Notebook:  Define a conditional distribution on segmental wordforms given an orthographic one.ipynb
Output Notebook: LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Define pW_V given LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 61/61 [00:23<00:00,  2.62it/s]


Input Notebook:  Define a conditional distribution on segmental wordforms given an orthographic one.ipynb
Output Notebook: LTR_Buckeye_aligned_w_GD_AmE_destressed/Define pW_V given LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 61/61 [00:04<00:00, 12.80it/s]


# Step 4: Pre-calculate remaining forward model components and meta-data

Note that none of these steps need actually be ordered with respect to each other: the ordering below is arbitrary.

## Step 4a: Generate triphone lexicon distributions for every triphone channel model

In [53]:
def get_immediate_subdirectories(a_dir):
    return [name for name in listdir(a_dir)
            if path.isdir(path.join(a_dir, name))]

In [54]:
subdirs = get_immediate_subdirectories('.')
len(subdirs)

34

In [55]:
channel_model_fps = []
for d in subdirs:
    files = listdir(d)
    is_triph_channel_model = lambda fn: 'pY1X0X1X2.json' in fn
    CM_files = list(filter(is_triph_channel_model,
                           files))
    for CM_file in CM_files:
        channel_model_fps.append(path.join(d, CM_file))
len(channel_model_fps)
channel_model_fps

23

['CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_unaligned_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_

In [56]:
triph_lex_bundles = []
for cm_fp in channel_model_fps:
    bundle = dict()
    bundle['c'] = cm_fp
    bundle['output_dir'] = path.dirname(cm_fp)
    bundle['c_fn'] = path.basename(cm_fp)
    bundle['o_fn_prefix'] = bundle['c_fn'].split('pY1X0X1X2')[0] + 'pX0X1X2'
    bundle['o'] = path.join(bundle['output_dir'],
                            bundle['o_fn_prefix'])
    bundle['nb_fp'] = path.join(bundle['output_dir'],
                                f"Generating {bundle['c_fn'].split('pY1X0X1X2.json')[0][:-1]} uniform triphone lexicon dist.ipynb")
    bundle
    print(' ')
    triph_lex_bundles.append(bundle)

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05',
 'c_fn': 'pY1X0X1X2.json',
 'o_fn_prefix': 'pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Generating  uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05',
 'c_fn': 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'o_fn_prefix': 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Generating LTR_newdic_destressed_aligned_CM_filtered_LM_filtered uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1',
 'c_fn': 'pY1X0X1X2.json',
 'o_fn_prefix': 'pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Generating  uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1',
 'c_fn': 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'o_fn_prefix': 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Generating LTR_newdic_destressed_aligned_CM_filtered_LM_filtered uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01',
 'c_fn': 'pY1X0X1X2.json',
 'o_fn_prefix': 'pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/Generating  uniform triphone lexicon dist.ipynb'}

...

{'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05',
 'c_fn': 'pY1X0X1X2.json',
 'o_fn_prefix': 'pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Generating  uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05',
 'c_fn': 'LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'o_fn_prefix': 'LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Generating LTR_CMU_destressed_aligned_CM_filtered_LM_filtered uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05',
 'c_fn': 'pY1X0X1X2.json',
 'o_fn_prefix': 'pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Generating  uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05',
 'c_fn': 'LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'o_fn_prefix': 'LTR_Buckeye_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Generating LTR_Buckeye_aligned_CM_filtered_LM_filtered uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_unaligned_pseudocount0.05/pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_unaligned_pseudocount0.05',
 'c_fn': 'pY1X0X1X2.json',
 'o_fn_prefix': 'pX0X1X2',
 'o': 'CM_AmE_destressed_unaligned_pseudocount0.05/pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_unaligned_pseudocount0.05/Generating  uniform triphone lexicon dist.ipynb'}

In [65]:
# takes about ~1m on wittgenstein
for bundle in triph_lex_bundles:
    output_dir = bundle['output_dir']
    if not path.exists(output_dir):
        print(f"Making output path {output_dir}")
        makedirs(output_dir)
    
    nb = pm.execute_notebook(
    'Generate triphone lexicon distribution from channel model.ipynb',
    bundle['nb_fp'],
    parameters=dict(c = bundle['c'],
                    o = bundle['o'])
    )
    print('\n')

Input Notebook:  Generate triphone lexicon distribution from channel model.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Generating  uniform triphone lexicon dist.ipynb
100%|██████████| 23/23 [00:01<00:00, 11.73it/s]


Input Notebook:  Generate triphone lexicon distribution from channel model.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Generating LTR_newdic_destressed_aligned_CM_filtered_LM_filtered uniform triphone lexicon dist.ipynb
100%|██████████| 23/23 [00:00<00:00, 32.38it/s]


Input Notebook:  Generate triphone lexicon distribution from channel model.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Generating  uniform triphone lexicon dist.ipynb
100%|██████████| 23/23 [00:01<00:00, 11.91it/s]


Input Notebook:  Generate triphone lexicon distribution from channel model.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Generating LTR_newdic_destressed_aligned_CM_filtered_LM_filtered uniform triphone lexicon dist.ipynb
100%|██████████| 23/23 [00:00<00:00, 33.70it/s]


Input Notebook:  Generate triphone lexicon distribution from channel model.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/Generating  uniform triphone lexicon dist.ipynb
100%|██████████| 23/23 [00:01<00:00, 11.74it/s]


...

Input Notebook:  Generate triphone lexicon distribution from channel model.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Generating  uniform triphone lexicon dist.ipynb
100%|██████████| 23/23 [00:01<00:00, 12.16it/s]


Input Notebook:  Generate triphone lexicon distribution from channel model.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Generating LTR_CMU_destressed_aligned_CM_filtered_LM_filtered uniform triphone lexicon dist.ipynb
100%|██████████| 23/23 [00:00<00:00, 28.94it/s]


Input Notebook:  Generate triphone lexicon distribution from channel model.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Generating  uniform triphone lexicon dist.ipynb
100%|██████████| 23/23 [00:01<00:00, 11.92it/s]


Input Notebook:  Generate triphone lexicon distribution from channel model.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Generating LTR_Buckeye_aligned_CM_filtered_LM_filtered uniform triphone lexicon dist.ipynb
100%|██████████| 23/23 [00:00<00:00, 37.42it/s]


Input Notebook:  Generate triphone lexicon distribution from channel model.ipynb
Output Notebook: CM_AmE_destressed_unaligned_pseudocount0.05/Generating  uniform triphone lexicon dist.ipynb
100%|██████████| 23/23 [00:02<00:00,  9.04it/s]


## Step 4b: Pre-calculate prefix relation, $k$-cousins, and $k$-spheres for each segmental lexicon

(This is comparable to the `Metadata` generation step in 2a above.)

In [57]:
pW_V_fps = [each['pW_V_fp'] + '.pW_V.json' for each in pW_V_fp_bundles]
pW_V_fps

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json']

In [156]:
# pW_V_fps = []
# for d in subdirs:
#     files = listdir(d)
#     is_pW_V = lambda fn: 'pW_V.json' in fn
#     pW_V_files = list(filter(is_pW_V,
#                            files))
#     for pW_V_file in pW_V_files:
#         pW_V_fps.append(path.join(d, pW_V_file))
# len(pW_V_fps)
# pW_V_fps

In [58]:
lexicon_md_bundles = []
for pW_V_fp in pW_V_fps:
    output_dir = bundle['o']
    if not path.exists(output_dir):
        print(f"Making output path '{output_dir}'")
        makedirs(output_dir)
    
    bundle = dict()
    bundle['p'] = pW_V_fp
    bundle['o'] = path.dirname(pW_V_fp)
    bundle['lex_name'] = path.basename(pW_V_fp).split('.pW_V.json')[0]
    bundle['nb_fp'] = path.join(bundle['o'],
                                f"Calculate prefix data, k-cousins, and k-spheres for {bundle['lex_name']}.ipynb")
    bundle
    print(' ')
    lexicon_md_bundles.append(bundle)

{'p': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed',
 'lex_name': 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered',
 'nb_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Calculate prefix data, k-cousins, and k-spheres for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'p': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed',
 'lex_name': 'LTR_CMU_destressed_aligned_CM_filtered_LM_filtered',
 'nb_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Calculate prefix data, k-cousins, and k-spheres for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'p': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'LTR_Buckeye_aligned_w_GD_AmE_destressed',
 'lex_name': 'LTR_Buckeye_aligned_CM_filtered_LM_filtered',
 'nb_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/Calculate prefix data, k-cousins, and k-spheres for LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}

In [59]:
# with J=-1, heavy CPU load, and no .npz export
#  - newdic CM_filtered_LM_filtered takes ~20-30m
#  - CMU CM_filtered_LM_filtered takes ~3.5-3.75h
# - Buckeye CM_filtered_LM_filtered takes ~10-15m

# with J=-1, no CPU load, *and* .npz export
#  - newdic CM_filtered_LM_filtered takes ~45m
#  - CMU CM_filtered_LM_filtered takes ~20.5h
# - Buckeye CM_filtered_LM_filtered takes ~19.5h
# (.npz representations are up to 20x smaller on disk 
#  and are significantly smaller in memory when loaded)
for bundle in lexicon_md_bundles:
    output_dir = bundle['o']
    if not path.exists(output_dir):
        print(f"Making output path {output_dir}")
        makedirs(output_dir)
    
    nb = pm.execute_notebook(
    'Calculate prefix data, k-cousins, and k-spheres.ipynb',
    bundle['nb_fp'],
    parameters=dict(p = bundle['p'],
                    o = bundle['o'])
    )
    print('\n')

Input Notebook:  Calculate prefix data, k-cousins, and k-spheres.ipynb
Output Notebook: LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Calculate prefix data, k-cousins, and k-spheres for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 171/171 [45:52<00:00,  1.95s/it]   


Input Notebook:  Calculate prefix data, k-cousins, and k-spheres.ipynb
Output Notebook: LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Calculate prefix data, k-cousins, and k-spheres for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 171/171 [21:02:49<00:00, 67.45s/it]       


Input Notebook:  Calculate prefix data, k-cousins, and k-spheres.ipynb
Output Notebook: LTR_Buckeye_aligned_w_GD_AmE_destressed/Calculate prefix data, k-cousins, and k-spheres for LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 171/171 [19:26<00:00,  1.02it/s] 


## Step 4c: Calculate the marginal probability $p(W|C)$ of each segmental wordform $w$ given $n$-gram contexts $C$

In [ ]:
#gather pV_C, pW_V fp pairs as numpy arrays

In [60]:
def LTR_to_pW_Vs(LTR_fp):
    matching_bundles = list(filter(lambda bundle: bundle['LTR_fp'] == LTR_fp,
                                   pW_V_fp_bundles))
    matching_pW_V_fps = set(map(lambda bundle: bundle['pW_V_fp'],
                                matching_bundles))
    return matching_pW_V_fps

In [61]:
pW_V_fp_bundles

[{'LTR_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered',
  'nb_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Define pW_V given LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'},
 {'LTR_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered',
  'nb_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Define pW_V given LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'},
 {'LTR_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered',
  'nb_fp': 'LTR_Buckeye_aligned_w_GD_AmE_de

In [62]:
def LTR_to_LD(LTR_fp):
    matching_bundles = list(filter(lambda bundle: bundle['l'] == LTR_fp,
                                   LD_projection_args))
    matching_pV_C_fps = set(map(lambda bundle: bundle['o'],
                                matching_bundles))
    return matching_pV_C_fps

In [63]:
LD_projection_args

[{'d': 'LD_Fisher_vocab_in_Buckeye_contexts/LD_fisher_vocab_in_buckeye_contexts.pV_C',
  'v': 'LM_Fisher/fisher_vocabulary_main.txt',
  'c': 'buckeye_contexts.txt',
  'l': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
  'o': 'LD_Fisher_vocab_in_Buckeye_contexts/LD_fisher_vocab_in_buckeye_contexts_projected_LTR_Buckeye.pV_C',
  'f': 'True',
  'nb_fp': 'LD_Fisher_vocab_in_Buckeye_contexts/Filter LD_fisher_vocab_in_buckeye_contexts against LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'},
 {'d': 'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts.pV_C',
  'v': 'LM_Fisher/fisher_vocabulary_main.txt',
  'c': 'swbd2003_contexts.txt',
  'l': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'o': 'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts_projected_LTR_CMU_destressed.pV_C',
  'f': 'True',
  'nb_fp': 'LD_Fisher_vocab_in_swbd2003_contexts

In [64]:
def get_matched_pW_V_LD_pairs(LTR_fp):
    matching_pW_V_fps = LTR_to_pW_Vs(LTR_fp)
    matching_pV_C_fps = LTR_to_LD(LTR_fp)
    
    return set(product(matching_pW_V_fps,
                       matching_pV_C_fps))

my_LTR_fps = list(map(lambda bundle: bundle['LTR_fp'],
                      pW_V_fp_bundles))

matched_pW_V_LD_pairs = union(map(get_matched_pW_V_LD_pairs,
                                  my_LTR_fps))
matched_pW_V_LD_pairs

{('LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered',
  'LD_Fisher_vocab_in_Buckeye_contexts/LD_fisher_vocab_in_buckeye_contexts_projected_LTR_Buckeye.pV_C'),
 ('LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered',
  'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts_projected_LTR_CMU_destressed.pV_C'),
 ('LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered',
  'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts_projected_LTR_newdic_destressed.pV_C')}

In [65]:
WD_bundles = []
for w,d in matched_pW_V_LD_pairs:
    bundle = dict()
    
    LTR_key = path.basename(w)
    LD_key = path.basename(d)
    C_key = LD_key.split('LD_fisher_vocab')[1].split('_projected_')[0]
    
    output_dir = path.dirname(d)
    output_prefix = LTR_key + C_key + '.pW_C'
    
    bundle['d'] = d + '.npy'
    bundle['w'] = w + '.pW_V.npz'
    
    bundle['o'] = path.join(output_dir, output_prefix)
    
    bundle['nb_fp'] = path.join(output_dir, f"Calculate segmental wordform distribution for {LTR_key}{C_key.replace('_', ' ')}.ipynb")
    
    bundle
    WD_bundles.append(bundle)

{'d': 'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts_projected_LTR_newdic_destressed.pV_C.npy',
 'w': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_swbd2003_contexts/Calculate segmental wordform distribution for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered in swbd2003 contexts.ipynb'}

{'d': 'LD_Fisher_vocab_in_Buckeye_contexts/LD_fisher_vocab_in_buckeye_contexts_projected_LTR_Buckeye.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_buckeye_contexts.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_contexts/Calculate segmental wordform distribution for LTR_Buckeye_aligned_CM_filtered_LM_filtered in buckeye contexts.ipynb'}

{'d': 'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts_projected_LTR_CMU_destressed.pV_C.npy',
 'w': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_swbd2003_contexts/Calculate segmental wordform distribution for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered in swbd2003 contexts.ipynb'}

In [66]:
# takes ≈5-10m on wittgenstein with no background load
#  ≈8-10m for filtered LTR_CMU_destressed in swbd2003 contexts
#  ≤30s for filtered LTR_Buckeye in buckeye contexts
#  ≈2-3m for filtered LTR_newdic_destressed in swbd2003 contexts
for bundle in WD_bundles:
    ensure_dir_exists(path.dirname(bundle['o']))
    
    nb = pm.execute_notebook(
    'Calculate segmental wordform distribution given corpus contexts.ipynb',
    bundle['nb_fp'],
    parameters=dict(d = bundle['d'],
                    w = bundle['w'],
                    o = bundle['o'])
    )
    print('\n')

Input Notebook:  Calculate segmental wordform distribution given corpus contexts.ipynb
Output Notebook: LD_Fisher_vocab_in_swbd2003_contexts/Calculate segmental wordform distribution for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered in swbd2003 contexts.ipynb
100%|██████████| 30/30 [02:37<00:00, 10.52s/it]


Input Notebook:  Calculate segmental wordform distribution given corpus contexts.ipynb
Output Notebook: LD_Fisher_vocab_in_Buckeye_contexts/Calculate segmental wordform distribution for LTR_Buckeye_aligned_CM_filtered_LM_filtered in buckeye contexts.ipynb
100%|██████████| 30/30 [00:17<00:00,  1.74it/s]


Input Notebook:  Calculate segmental wordform distribution given corpus contexts.ipynb
Output Notebook: LD_Fisher_vocab_in_swbd2003_contexts/Calculate segmental wordform distribution for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered in swbd2003 contexts.ipynb
100%|██████████| 30/30 [08:27<00:00, 16.93s/it]


## Step 4d: Define channel distributions on a set of segmental wordforms(+prefixes)

In [67]:
# gather paired (....pW_V.json, ...pY1X0X1X2.json) p(W|V), p(Y_i|X_{i-1},X_i;X_{i+1}) distributions
# output complete wordform channel models into the channel model directory with the same prefix that's on the triphone channel distribution file

In [68]:
# def LTR_to_pW_Vs(LTR_fp):
#     matching_bundles = list(filter(lambda bundle: bundle['LTR_fp'] == LTR_fp,
#                                    pW_V_fp_bundles))
#     matching_pW_V_fps = set(map(lambda bundle: bundle['pW_V_fp'],
#                                 matching_bundles))
#     return matching_pW_V_fps

def LTR_to_TCMs(LTR_fp):
    matching_bundles = list(filter(lambda bundle: bundle['l'] == LTR_fp,
                                   trimmed_LTR_CM_triples))
    matching_TCM_fps = set(map(lambda bundle: bundle['o'],
                               matching_bundles))
    return matching_TCM_fps

def matched_pW_Vs_and_TCMs(LTR_fp):
    matching_pW_V_fps = LTR_to_pW_Vs(LTR_fp)
    matching_TCM_fps = LTR_to_TCMs(LTR_fp)
    return {'LW_V_fps':matching_pW_V_fps,
            'TCM_fps':matching_TCM_fps}

def get_matched_pairs(LTR_fp):
    matching_TCM_fps = LTR_to_TCMs(LTR_fp)
    matching_pW_V_fps = LTR_to_pW_Vs(LTR_fp)
    matched_pairs = set(product(matching_TCM_fps,
                                matching_pW_V_fps))
    return matched_pairs

my_LTR_fps = list(map(lambda bundle: bundle['LTR_fp'],
                      pW_V_fp_bundles))

LCM_bundles = []
for c,w in union(map(get_matched_pairs,
                     my_LTR_fps)):
    bundle = dict()
    bundle['c'] = c
    bundle['w'] = w + '.pW_V.json'
    output_dir = path.dirname(c)
    output_prefix = path.basename(c).split('pY1X0X1X2.json')[0]
    bundle['o'] = path.join(output_dir, output_prefix)
    
    bundle['nb_fp'] = path.join(output_dir, f'Calculate wordform channel matrices for {path.basename(w)}.ipynb')
    
    bundle
    LCM_bundles.append(bundle)

{'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/Calculate wordform channel matrices for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/Calculate wordform channel matrices for LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/Calculate wordform channel matrices for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Calculate wordform channel matrices for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/LTR_Buckeye_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/Calculate wordform channel matrices for LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Calculate wordform channel matrices for LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Calculate wordform channel matrices for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Calculate wordform channel matrices for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/Calculate wordform channel matrices for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

In [69]:
# takes ~2m on wittgenstein with no background load and no .npz files
# takes ~12m on wittgenstein with no background load *and* .npz files
for bundle in LCM_bundles:
    ensure_dir_exists(path.dirname(bundle['o']))
    
    nb = pm.execute_notebook(
    'Calculate segmental wordform and prefix channel matrices.ipynb',
    bundle['nb_fp'],
    parameters=dict(c = bundle['c'],
                    w = bundle['w'],
                    o = bundle['o'],
                    f = 'False')
    )
    print('\n')

Input Notebook:  Calculate segmental wordform and prefix channel matrices.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/Calculate wordform channel matrices for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 86/86 [03:09<00:00,  2.25s/it]


Input Notebook:  Calculate segmental wordform and prefix channel matrices.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/Calculate wordform channel matrices for LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 86/86 [00:22<00:00,  3.88it/s]


Input Notebook:  Calculate segmental wordform and prefix channel matrices.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/Calculate wordform channel matrices for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 86/86 [03:01<00:00,  2.54s/it]


Input Notebook:  Calculate segmental wordform and prefix channel matrices.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Calculate wordform channel matrices for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 86/86 [03:09<00:00,  2.20s/it]


Input Notebook:  Calculate segmental wordform and prefix channel matrices.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/Calculate wordform channel matrices for LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 86/86 [00:21<00:00,  3.92it/s]


Input Notebook:  Calculate segmental wordform and prefix channel matrices.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Calculate wordform channel matrices for LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 86/86 [00:21<00:00,  3.96it/s]


Input Notebook:  Calculate segmental wordform and prefix channel matrices.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Calculate wordform channel matrices for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 86/86 [00:28<00:00,  1.89it/s]


Input Notebook:  Calculate segmental wordform and prefix channel matrices.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Calculate wordform channel matrices for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 86/86 [00:28<00:00,  1.88it/s]


Input Notebook:  Calculate segmental wordform and prefix channel matrices.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/Calculate wordform channel matrices for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb
100%|██████████| 86/86 [00:28<00:00,  1.92it/s]


# Step 5: Calculate posterior probabilities

## Step 5a: Calculate $p(V|W, C)$

$p(\hat{V} = v^*|\hat{X}_0^f = x_0^{'f}, c) = \frac{p(x_0^{'f}|v^*)p(v^*|c)}{p(x_0^{'f}|c)}$ 

In [70]:
#gather aligned triples of filepaths defining
# - $p(V|C)$
# - $p(W|C)$
# - $p(W|V)$
# construct the output filename and location (probably in LD?)
# construct output notebook filepaths

In [71]:
posterior_WD_bundles = []
for bundle in WD_bundles:
    output_dir = path.dirname(bundle['d'])
    LTR_key = path.basename(bundle['w']).split('.pW_V.npz')[0]
    contexts_key = path.basename(bundle['o']).split('LM_filtered')[1].split('.pW_C')[0].replace('_', ' ')
    output_base_prefix = LTR_key + contexts_key.replace(' ', '_') + '.pV_WC'
    
    new_bundle = dict()
    new_bundle['d'] = bundle['d']          #p(V|C) as .npy
    new_bundle['w'] = bundle['w']          #p(W|V) as .npz
    new_bundle['m'] = bundle['o'] + '.npy' #p(W|C) as .npy
    new_bundle['o'] = path.join(output_dir, output_base_prefix)
    new_bundle['nb_fp'] = path.join(output_dir, f'Calculate orthographic posterior given segmental wordform + context for {LTR_key}{contexts_key}' + '.ipynb')
    new_bundle
    posterior_WD_bundles.append(new_bundle)
    print('\n')

{'d': 'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts_projected_LTR_newdic_destressed.pV_C.npy',
 'w': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_swbd2003_contexts/Calculate orthographic posterior given segmental wordform + context for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered in swbd2003 contexts.ipynb'}

{'d': 'LD_Fisher_vocab_in_Buckeye_contexts/LD_fisher_vocab_in_buckeye_contexts_projected_LTR_Buckeye.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_buckeye_contexts.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_buckeye_contexts.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_contexts/Calculate orthographic posterior given segmental wordform + context for LTR_Buckeye_aligned_CM_filtered_LM_filtered in buckeye contexts.ipynb'}

{'d': 'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts_projected_LTR_CMU_destressed.pV_C.npy',
 'w': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_swbd2003_contexts/Calculate orthographic posterior given segmental wordform + context for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered in swbd2003 contexts.ipynb'}

In [ ]:
# takes ?m for wittgenstein w/ no background load and J=-1
for bundle in posterior_WD_bundles:
    output_dir = path.dirname(bundle['o'])
    ensure_dir_exists(output_dir)
    
    pm.execute_notebook(
    'Calculate orthographic posterior given segmental wordform + context.ipynb',
    bundle['nb_fp'],
    parameters=dict(d = bundle['d'],
                   w = bundle['w'],
                   m = bundle['m'],
                   o = bundle['o'])
    )
    print('\n')

Input Notebook:  Calculate orthographic posterior given segmental wordform + context.ipynb
Output Notebook: LD_Fisher_vocab_in_swbd2003_contexts/Calculate orthographic posterior given segmental wordform + context for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered in swbd2003 contexts.ipynb


  0%|          | 0/44 [00:00<?, ?it/s]

 11%|█▏        | 5/44 [00:00<00:00, 48.77it/s]

 25%|██▌       | 11/44 [00:00<00:00, 50.21it/s]

 32%|███▏      | 14/44 [00:01<00:05,  5.36it/s]

 41%|████      | 18/44 [00:04<00:07,  3.26it/s]

 41%|████      | 18/44 [00:16<00:07,  3.26it/s]

 43%|████▎     | 19/44 [01:20<09:35, 23.00s/it]

 45%|████▌     | 20/44 [01:20<06:27, 16.15s/it]

 48%|████▊     | 21/44 [02:35<12:59, 33.88s/it]

 57%|█████▋    | 25/44 [02:35<07:30, 23.73s/it]

 66%|██████▌   | 29/44 [02:39<04:12, 16.85s/it]

 73%|███████▎  | 32/44 [02:39<02:21, 11.81s/it]

 84%|████████▍ | 37/44 [02:39<00:57,  8.27s/it]

 84%|████████▍ | 37/44 [02:56<00:57,  8.27s/it]

## Step 5b: Calculate $p(\hat{X}_0^f|X_0^f, C)$

Given a choice of parameters $\epsilon$ and $n$, and given
 - wordform channel matrices $p(Y_0^f|X_0^f)$
 - a contextual distribution on segmental wordforms $p(X_0^f|C)$
 - segmental lexicon metadata pre-calculating $k$-cousins/$k$-spheres up to $k=4$
 
Calculate

$$\hat{p}(\hat{X}_0^f = x_0^{'f}|X_0^f = x_0^{*f}, c) = \frac{1}{n} \sum\limits_{y_0^f \in S} p(\hat{X}_0^f = x_0^{'f}|y_0^f, c)$$
 where 
  - edit distance $d(x_0^{'f}, x_0^{*f}) \leq 4$
  - $S = $ a set of $n$ samples from $p(Y_0^f|x_0^{*f})$. In practice an $n \approx 50$ seems to result in estimates that are within $10^{-6}$ of the true estimate. 
  - $p(\hat{X}_0^f = x_0^{'f}|Y_0^f = y_0^f, c) = \frac{p(y_0^f|x_0^{'f})p(x_0^{'f}|c)}{p(y_0^f | c)}$
  - $p(y_0^f| c) = \sum\limits_{v', x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|v')p(v'|c) = \sum\limits_{x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|c)$

In [ ]:
# where do wordform channel matrices live?
#  where are they bundled?
#   - LCM_bundles
# where do contextual distributions on segmental wordforms live?
#   where are they bundled?
#   - WD_bundles
# where do lexicon metadata live?
#   where are they bundled?
#   - lexicon_md_bundles

In [ ]:
'Calculate segmental posterior given segmental wordform + context'

In [ ]:
#plan:
# 1. Check if there are components of the posterior calculation that are worth pre-calculating; if so, calculate them.
#    - e.g. pW_C, pV_W-hat,C
# 2. Next notebook should have a flag for calculating just p(V-hat = v* | V = v*) ∀v* vs. the full p(V-hat | V = v*) ∀v*
#    - Consider facilitating SLURM cluster jobs by 
#       - making posterior calculation deterministic (unlike P4bnt2) 
#       - adding/supporting optional notebook arguments that specify which parts of the posterior dist to calculate

## Step 5c: Calculate $p(\hat{V} = v^*| V = v^*, C)$

In [ ]:
#FIXME

# Step 6: Generating analysis measures